In [6]:
#!/usr/bin/env python3
"""
BLOSUM Matrix Analysis and Visualization
Creates similarity matrices and heatmaps for sequence comparison
"""

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from pathlib import Path


# BLOSUM62 Matrix (most commonly used for protein sequence comparison)
BLOSUM62 = {
    'A': {'A': 4, 'R': -1, 'N': -2, 'D': -2, 'C': 0, 'Q': -1, 'E': -1, 'G': 0, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 0, 'W': -3, 'Y': -2, 'V': 0},
    'R': {'A': -1, 'R': 5, 'N': 0, 'D': -2, 'C': -3, 'Q': 1, 'E': 0, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 2, 'M': -1, 'F': -3, 'P': -2, 'S': -1, 'T': -1, 'W': -3, 'Y': -2, 'V': -3},
    'N': {'A': -2, 'R': 0, 'N': 6, 'D': 1, 'C': -3, 'Q': 0, 'E': 0, 'G': 0, 'H': 1, 'I': -3, 'L': -3, 'K': 0, 'M': -2, 'F': -3, 'P': -2, 'S': 1, 'T': 0, 'W': -4, 'Y': -2, 'V': -3},
    'D': {'A': -2, 'R': -2, 'N': 1, 'D': 6, 'C': -3, 'Q': 0, 'E': 2, 'G': -1, 'H': -1, 'I': -3, 'L': -4, 'K': -1, 'M': -3, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -4, 'Y': -3, 'V': -3},
    'C': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': 9, 'Q': -3, 'E': -4, 'G': -3, 'H': -3, 'I': -1, 'L': -1, 'K': -3, 'M': -1, 'F': -2, 'P': -3, 'S': -1, 'T': -1, 'W': -2, 'Y': -2, 'V': -1},
    'Q': {'A': -1, 'R': 1, 'N': 0, 'D': 0, 'C': -3, 'Q': 5, 'E': 2, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 1, 'M': 0, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -2, 'Y': -1, 'V': -2},
    'E': {'A': -1, 'R': 0, 'N': 0, 'D': 2, 'C': -4, 'Q': 2, 'E': 5, 'G': -2, 'H': 0, 'I': -3, 'L': -3, 'K': 1, 'M': -2, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -2},
    'G': {'A': 0, 'R': -2, 'N': 0, 'D': -1, 'C': -3, 'Q': -2, 'E': -2, 'G': 6, 'H': -2, 'I': -4, 'L': -4, 'K': -2, 'M': -3, 'F': -3, 'P': -2, 'S': 0, 'T': -2, 'W': -2, 'Y': -3, 'V': -3},
    'H': {'A': -2, 'R': 0, 'N': 1, 'D': -1, 'C': -3, 'Q': 0, 'E': 0, 'G': -2, 'H': 8, 'I': -3, 'L': -3, 'K': -1, 'M': -2, 'F': -1, 'P': -2, 'S': -1, 'T': -2, 'W': -2, 'Y': 2, 'V': -3},
    'I': {'A': -1, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -3, 'E': -3, 'G': -4, 'H': -3, 'I': 4, 'L': 2, 'K': -3, 'M': 1, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -3, 'Y': -1, 'V': 3},
    'L': {'A': -1, 'R': -2, 'N': -3, 'D': -4, 'C': -1, 'Q': -2, 'E': -3, 'G': -4, 'H': -3, 'I': 2, 'L': 4, 'K': -2, 'M': 2, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -2, 'Y': -1, 'V': 1},
    'K': {'A': -1, 'R': 2, 'N': 0, 'D': -1, 'C': -3, 'Q': 1, 'E': 1, 'G': -2, 'H': -1, 'I': -3, 'L': -2, 'K': 5, 'M': -1, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -2},
    'M': {'A': -1, 'R': -1, 'N': -2, 'D': -3, 'C': -1, 'Q': 0, 'E': -2, 'G': -3, 'H': -2, 'I': 1, 'L': 2, 'K': -1, 'M': 5, 'F': 0, 'P': -2, 'S': -1, 'T': -1, 'W': -1, 'Y': -1, 'V': 1},
    'F': {'A': -2, 'R': -3, 'N': -3, 'D': -3, 'C': -2, 'Q': -3, 'E': -3, 'G': -3, 'H': -1, 'I': 0, 'L': 0, 'K': -3, 'M': 0, 'F': 6, 'P': -4, 'S': -2, 'T': -2, 'W': 1, 'Y': 3, 'V': -1},
    'P': {'A': -1, 'R': -2, 'N': -2, 'D': -1, 'C': -3, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -3, 'L': -3, 'K': -1, 'M': -2, 'F': -4, 'P': 7, 'S': -1, 'T': -1, 'W': -4, 'Y': -3, 'V': -2},
    'S': {'A': 1, 'R': -1, 'N': 1, 'D': 0, 'C': -1, 'Q': 0, 'E': 0, 'G': 0, 'H': -1, 'I': -2, 'L': -2, 'K': 0, 'M': -1, 'F': -2, 'P': -1, 'S': 4, 'T': 1, 'W': -3, 'Y': -2, 'V': -2},
    'T': {'A': 0, 'R': -1, 'N': 0, 'D': -1, 'C': -1, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 5, 'W': -2, 'Y': -2, 'V': 0},
    'W': {'A': -3, 'R': -3, 'N': -4, 'D': -4, 'C': -2, 'Q': -2, 'E': -3, 'G': -2, 'H': -2, 'I': -3, 'L': -2, 'K': -3, 'M': -1, 'F': 1, 'P': -4, 'S': -3, 'T': -2, 'W': 11, 'Y': 2, 'V': -3},
    'Y': {'A': -2, 'R': -2, 'N': -2, 'D': -3, 'C': -2, 'Q': -1, 'E': -2, 'G': -3, 'H': 2, 'I': -1, 'L': -1, 'K': -2, 'M': -1, 'F': 3, 'P': -3, 'S': -2, 'T': -2, 'W': 2, 'Y': 7, 'V': -1},
    'V': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -2, 'E': -2, 'G': -3, 'H': -3, 'I': 3, 'L': 1, 'K': -2, 'M': 1, 'F': -1, 'P': -2, 'S': -2, 'T': 0, 'W': -3, 'Y': -1, 'V': 4}
}


# CLUSTAL ALIGNMENT WITH ANCESTOR
CLUSTAL_ALIGNMENT = """
CLUSTAL O(1.2.4) multiple sequence alignment

protein_mousecys    PQKSKVDXNKGVTGTVYEYGANTIDGGEFVNFQQYAGKXILFVNVASFCGLTATYPELNT	60
protein_ancestor    SQKMKMDCYKGVTGTIYEYGALTLNGEEYIQFKQYAGKHVLFINVATY-GLTAQYPELNA	59
protein_humansec    PQNRKVDXNKGVTGTIYEYGALTLNGEEYIQFKQFAGKXVLFVNVAAYUGLAAQYPELNA	60

protein_mousecys    LQEELKPFNVTVLGFPCNQFGKQEPGKNSEILLGLKYVRPGGGYVPNFQLFEKGDVNGDN	120
protein_ancestor    LQEELKPFGVVLLGFPCNQFGKQEPGKNSEILSGLKYVRPGGGFVPNFQLFEKGDVNGEK	119
protein_humansec    LQEELKNFGVIVLAFPCNQFGKQEPGTNSEILLGLKYVCPGSGFVPSFQLFEKGDVNGEK	120

protein_mousecys    EQKVFSFLKNSXPPTSELFGSPEXLFWDPMKVXDIRWNFEKFLVGPDGVPVMRWFXXTPV	180
protein_ancestor    EQKVFTFLKNSCPPTSDLLGSPKQLFWEPMKVHDIRWNFEKFLVGPDGVPVMRWFHRASV	179
protein_humansec    EQKVFTFLKNSXPPTSDLLGSSSQLFWEPMKVXDIRWNFEKFLVGPDGVPVMXWFXQAPV	180

protein_mousecys    RIVQSDIMEYLNQTS--	195
protein_ancestor    STVKSDILEYLKQFTPE	196
protein_humansec    STVKSDILEYLKQFNTX	197
"""


def extract_sequences_from_clustal(alignment_text):
    """Extract sequences from CLUSTAL alignment."""
    sequences = {}
    for line in alignment_text.strip().split('\n'):
        line = line.strip()
        if not line or line.startswith('CLUSTAL') or line.startswith('*') or line.startswith(':') or line.startswith('.'):
            continue
        parts = line.split()
        if len(parts) >= 2:
            seq_name = parts[0]
            seq_fragment = parts[1]
            if seq_name not in sequences:
                sequences[seq_name] = ""
            sequences[seq_name] += seq_fragment
    return sequences


def calculate_blosum_score(seq1, seq2, blosum_matrix=BLOSUM62):
    """Calculate BLOSUM alignment score between two sequences."""
    score = 0
    comparisons = 0
    
    for aa1, aa2 in zip(seq1, seq2):
        # Skip gaps and unknown residues
        if aa1 in '-XU*' or aa2 in '-XU*':
            continue
        
        # Get BLOSUM score
        if aa1 in blosum_matrix and aa2 in blosum_matrix[aa1]:
            score += blosum_matrix[aa1][aa2]
            comparisons += 1
    
    # Normalize score
    if comparisons > 0:
        normalized_score = score / comparisons
    else:
        normalized_score = 0
    
    return score, normalized_score, comparisons


def create_similarity_matrix(sequences, blosum_matrix=BLOSUM62):
    """Create pairwise similarity matrix using BLOSUM scores."""
    seq_names = list(sequences.keys())
    n = len(seq_names)
    
    # Initialize matrices
    score_matrix = np.zeros((n, n))
    normalized_matrix = np.zeros((n, n))
    
    for i, name1 in enumerate(seq_names):
        for j, name2 in enumerate(seq_names):
            if i == j:
                # Self-comparison
                seq = sequences[name1]
                clean_seq = seq.replace('-', '').replace('X', '').replace('U', '')
                score, norm_score, _ = calculate_blosum_score(clean_seq, clean_seq, blosum_matrix)
                score_matrix[i][j] = score
                normalized_matrix[i][j] = norm_score
            else:
                score, norm_score, _ = calculate_blosum_score(
                    sequences[name1], sequences[name2], blosum_matrix
                )
                score_matrix[i][j] = score
                normalized_matrix[i][j] = norm_score
    
    return score_matrix, normalized_matrix, seq_names


def visualize_blosum_matrix(output_file="blosum62_matrix.png"):
    """Visualize the BLOSUM62 matrix itself."""
    
    amino_acids = sorted(BLOSUM62.keys())
    n = len(amino_acids)
    matrix = np.zeros((n, n))
    
    for i, aa1 in enumerate(amino_acids):
        for j, aa2 in enumerate(amino_acids):
            matrix[i][j] = BLOSUM62[aa1][aa2]
    
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Create heatmap
    im = ax.imshow(matrix, cmap='RdBu_r', aspect='auto', vmin=-4, vmax=11)
    
    # Set ticks
    ax.set_xticks(np.arange(n))
    ax.set_yticks(np.arange(n))
    ax.set_xticklabels(amino_acids, fontsize=10)
    ax.set_yticklabels(amino_acids, fontsize=10)
    
    # Rotate x labels
    plt.setp(ax.get_xticklabels(), rotation=0, ha="center")
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('BLOSUM62 Score', rotation=270, labelpad=20, fontsize=12)
    
    # Add values in cells
    for i in range(n):
        for j in range(n):
            text = ax.text(j, i, int(matrix[i, j]),
                          ha="center", va="center", color="black" if abs(matrix[i, j]) < 5 else "white",
                          fontsize=7)
    
    ax.set_title('BLOSUM62 Substitution Matrix', fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Amino Acid', fontsize=12, fontweight='bold')
    ax.set_ylabel('Amino Acid', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    print(f"✅ BLOSUM62 matrix visualization saved to: {output_file}")
    plt.close()


def visualize_sequence_similarity_matrix(sequences, output_file="sequence_similarity_matrix.png"):
    """Visualize pairwise similarity between sequences using BLOSUM scores."""
    
    score_matrix, normalized_matrix, seq_names = create_similarity_matrix(sequences)
    
    # Shorten sequence names for display
    display_names = []
    for name in seq_names:
        if 'ancestor' in name.lower():
            display_names.append('Ancestor')
        elif 'mouse' in name.lower():
            display_names.append('Mouse')
        elif 'human' in name.lower():
            display_names.append('Human')
        else:
            display_names.append(name[:15])
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Raw BLOSUM scores
    im1 = ax1.imshow(score_matrix, cmap='YlOrRd', aspect='auto')
    ax1.set_xticks(np.arange(len(display_names)))
    ax1.set_yticks(np.arange(len(display_names)))
    ax1.set_xticklabels(display_names, fontsize=11, fontweight='bold')
    ax1.set_yticklabels(display_names, fontsize=11, fontweight='bold')
    plt.setp(ax1.get_xticklabels(), rotation=45, ha="right")
    
    # Add values
    for i in range(len(display_names)):
        for j in range(len(display_names)):
            text = ax1.text(j, i, f'{score_matrix[i, j]:.0f}',
                           ha="center", va="center", 
                           color="white" if score_matrix[i, j] > score_matrix.max()/2 else "black",
                           fontsize=10, fontweight='bold')
    
    cbar1 = plt.colorbar(im1, ax=ax1)
    cbar1.set_label('Total BLOSUM Score', rotation=270, labelpad=20, fontsize=11)
    ax1.set_title('Raw BLOSUM Alignment Scores', fontsize=14, fontweight='bold', pad=15)
    
    # Plot 2: Normalized scores
    im2 = ax2.imshow(normalized_matrix, cmap='RdYlGn', aspect='auto', vmin=-2, vmax=4)
    ax2.set_xticks(np.arange(len(display_names)))
    ax2.set_yticks(np.arange(len(display_names)))
    ax2.set_xticklabels(display_names, fontsize=11, fontweight='bold')
    ax2.set_yticklabels(display_names, fontsize=11, fontweight='bold')
    plt.setp(ax2.get_xticklabels(), rotation=45, ha="right")
    
    # Add values
    for i in range(len(display_names)):
        for j in range(len(display_names)):
            text = ax2.text(j, i, f'{normalized_matrix[i, j]:.2f}',
                           ha="center", va="center",
                           color="white" if normalized_matrix[i, j] > 2 or normalized_matrix[i, j] < -1 else "black",
                           fontsize=10, fontweight='bold')
    
    cbar2 = plt.colorbar(im2, ax=ax2)
    cbar2.set_label('Average Score per Position', rotation=270, labelpad=20, fontsize=11)
    ax2.set_title('Normalized BLOSUM Scores', fontsize=14, fontweight='bold', pad=15)
    
    plt.suptitle('Sequence Similarity Matrix (BLOSUM62-based)', 
                 fontsize=16, fontweight='bold', y=1.02)
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    print(f"✅ Sequence similarity matrix saved to: {output_file}")
    plt.close()
    
    return score_matrix, normalized_matrix, seq_names


def analyze_position_specific_scores(sequences, output_file="position_specific_blosum.png"):
    """Analyze BLOSUM scores at each position of the alignment."""
    
    ancestor_key = None
    for key in sequences.keys():
        if 'ancestor' in key.lower():
            ancestor_key = key
            break
    
    if not ancestor_key:
        print("Warning: No ancestor sequence found")
        return
    
    ancestor_seq = sequences[ancestor_key]
    
    # Calculate position-specific scores for mouse and human
    mouse_key = [k for k in sequences.keys() if 'mouse' in k.lower()][0]
    human_key = [k for k in sequences.keys() if 'human' in k.lower()][0]
    
    mouse_seq = sequences[mouse_key]
    human_seq = sequences[human_key]
    
    positions = []
    mouse_scores = []
    human_scores = []
    
    for i, (anc_aa, mouse_aa, human_aa) in enumerate(zip(ancestor_seq, mouse_seq, human_seq)):
        if anc_aa in '-XU*':
            continue
        
        positions.append(i + 1)
        
        # Mouse score
        if mouse_aa in '-XU*' or mouse_aa not in BLOSUM62.get(anc_aa, {}):
            mouse_scores.append(0)
        else:
            mouse_scores.append(BLOSUM62[anc_aa][mouse_aa])
        
        # Human score
        if human_aa in '-XU*' or human_aa not in BLOSUM62.get(anc_aa, {}):
            human_scores.append(0)
        else:
            human_scores.append(BLOSUM62[anc_aa][human_aa])
    
    # Create plot
    fig, ax = plt.subplots(figsize=(16, 6))
    
    width = 0.4
    x = np.arange(len(positions))
    
    bars1 = ax.bar(x - width/2, mouse_scores, width, label='Mouse', 
                   color='blue', alpha=0.7, edgecolor='black')
    bars2 = ax.bar(x + width/2, human_scores, width, label='Human', 
                   color='red', alpha=0.7, edgecolor='black')
    
    # Add reference line at 0
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
    
    # Highlight conserved regions (both positive)
    for i, (m_score, h_score) in enumerate(zip(mouse_scores, human_scores)):
        if m_score > 0 and h_score > 0:
            ax.axvspan(i - 0.5, i + 0.5, alpha=0.1, color='green')
    
    ax.set_xlabel('Position in Alignment', fontsize=12, fontweight='bold')
    ax.set_ylabel('BLOSUM62 Score vs Ancestor', fontsize=12, fontweight='bold')
    ax.set_title('Position-Specific Similarity to Ancestor', fontsize=14, fontweight='bold', pad=20)
    ax.legend(fontsize=12, loc='upper right')
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Set x-axis to show every 10th position
    tick_positions = x[::10]
    tick_labels = [positions[i] for i in range(0, len(positions), 10)]
    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels)
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    print(f"✅ Position-specific BLOSUM analysis saved to: {output_file}")
    plt.close()


def create_blosum_summary_report(sequences, output_file="blosum_summary.txt"):
    """Create text summary of BLOSUM analysis."""
    
    score_matrix, normalized_matrix, seq_names = create_similarity_matrix(sequences)
    
    # Find ancestor index
    ancestor_idx = None
    for i, name in enumerate(seq_names):
        if 'ancestor' in name.lower():
            ancestor_idx = i
            break
    
    if ancestor_idx is None:
        print("Warning: No ancestor found")
        return
    
    with open(output_file, 'w') as f:
        f.write("="*80 + "\n")
        f.write("BLOSUM62 SIMILARITY ANALYSIS SUMMARY\n")
        f.write("="*80 + "\n\n")
        
        f.write("BLOSUM62 is a substitution matrix used to score protein sequence alignments.\n")
        f.write("Positive scores = similar amino acids (favorable substitution)\n")
        f.write("Negative scores = dissimilar amino acids (unfavorable substitution)\n\n")
        
        f.write("-"*80 + "\n")
        f.write("PAIRWISE SIMILARITY SCORES TO ANCESTOR\n")
        f.write("-"*80 + "\n\n")
        
        for i, name in enumerate(seq_names):
            if i == ancestor_idx:
                continue
            
            display_name = "Mouse" if 'mouse' in name.lower() else "Human" if 'human' in name.lower() else name
            
            f.write(f"{display_name}:\n")
            f.write(f"  Total BLOSUM Score: {score_matrix[i][ancestor_idx]:.1f}\n")
            f.write(f"  Normalized Score: {normalized_matrix[i][ancestor_idx]:.3f}\n")
            f.write(f"  (Higher = more similar to ancestor)\n\n")
        
        # Interpretation
        f.write("-"*80 + "\n")
        f.write("INTERPRETATION\n")
        f.write("-"*80 + "\n\n")
        
        mouse_idx = [i for i, n in enumerate(seq_names) if 'mouse' in n.lower()][0]
        human_idx = [i for i, n in enumerate(seq_names) if 'human' in n.lower()][0]
        
        mouse_score = normalized_matrix[mouse_idx][ancestor_idx]
        human_score = normalized_matrix[human_idx][ancestor_idx]
        
        if human_score > mouse_score:
            f.write(f"Human sequences show HIGHER BLOSUM scores ({human_score:.3f}) compared to\n")
            f.write(f"mouse sequences ({mouse_score:.3f}), indicating that human GPX6 has\n")
            f.write(f"more biochemically similar amino acids to the ancestor.\n\n")
            f.write(f"This suggests human GPX6 is more conserved in terms of amino acid\n")
            f.write(f"properties (size, charge, hydrophobicity) compared to the ancestor.\n")
        else:
            f.write(f"Mouse sequences show HIGHER BLOSUM scores than human sequences.\n")
    
    print(f"✅ BLOSUM summary report saved to: {output_file}")


def main():
    """Main function."""
    
    print("="*80)
    print("BLOSUM MATRIX ANALYSIS AND VISUALIZATION")
    print("="*80)
    
    # Extract sequences from alignment
    print("\n[1] Extracting sequences from alignment...")
    sequences = extract_sequences_from_clustal(CLUSTAL_ALIGNMENT)
    print(f"    ✓ Found {len(sequences)} sequences")
    
    # Visualize BLOSUM62 matrix itself
    print("\n[2] Creating BLOSUM62 matrix visualization...")
    visualize_blosum_matrix()
    
    # Create similarity matrix
    print("\n[3] Creating sequence similarity matrix...")
    visualize_sequence_similarity_matrix(sequences)
    
    # Position-specific analysis
    print("\n[4] Analyzing position-specific BLOSUM scores...")
    analyze_position_specific_scores(sequences)
    
    # Create summary report
    print("\n[5] Generating BLOSUM summary report...")
    create_blosum_summary_report(sequences)
    
    print("\n" + "="*80)
    print("✅ BLOSUM ANALYSIS COMPLETE")
    print("="*80)
    print("\nOutput files:")
    print("  • blosum62_matrix.png - Visualization of BLOSUM62 substitution matrix")
    print("  • sequence_similarity_matrix.png - Pairwise similarity between sequences")
    print("  • position_specific_blosum.png - BLOSUM scores at each position")
    print("  • blosum_summary.txt - Text summary and interpretation")


if __name__ == "__main__":
    main()

BLOSUM MATRIX ANALYSIS AND VISUALIZATION

[1] Extracting sequences from alignment...
    ✓ Found 3 sequences

[2] Creating BLOSUM62 matrix visualization...
✅ BLOSUM62 matrix visualization saved to: blosum62_matrix.png

[3] Creating sequence similarity matrix...
✅ Sequence similarity matrix saved to: sequence_similarity_matrix.png

[4] Analyzing position-specific BLOSUM scores...
✅ Position-specific BLOSUM analysis saved to: position_specific_blosum.png

[5] Generating BLOSUM summary report...
✅ BLOSUM summary report saved to: blosum_summary.txt

✅ BLOSUM ANALYSIS COMPLETE

Output files:
  • blosum62_matrix.png - Visualization of BLOSUM62 substitution matrix
  • sequence_similarity_matrix.png - Pairwise similarity between sequences
  • position_specific_blosum.png - BLOSUM scores at each position
  • blosum_summary.txt - Text summary and interpretation


In [7]:
#!/usr/bin/env python3
"""
Simple Sequence Identity Calculator
Determines which sequences are closest to the ancestor based on % identity
"""

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


# CLUSTAL ALIGNMENT WITH ANCESTOR (from your code)
CLUSTAL_ALIGNMENT = """
CLUSTAL O(1.2.4) multiple sequence alignment

protein_mousecys    PQKSKVDXNKGVTGTVYEYGANTIDGGEFVNFQQYAGKXILFVNVASFCGLTATYPELNT	60
protein_ancestor    SQKMKMDCYKGVTGTIYEYGALTLNGEEYIQFKQYAGKHVLFINVATY-GLTAQYPELNA	59
protein_humansec    PQNRKVDXNKGVTGTIYEYGALTLNGEEYIQFKQFAGKXVLFVNVAAYUGLAAQYPELNA	60

protein_mousecys    LQEELKPFNVTVLGFPCNQFGKQEPGKNSEILLGLKYVRPGGGYVPNFQLFEKGDVNGDN	120
protein_ancestor    LQEELKPFGVVLLGFPCNQFGKQEPGKNSEILSGLKYVRPGGGFVPNFQLFEKGDVNGEK	119
protein_humansec    LQEELKNFGVIVLAFPCNQFGKQEPGTNSEILLGLKYVCPGSGFVPSFQLFEKGDVNGEK	120

protein_mousecys    EQKVFSFLKNSXPPTSELFGSPEXLFWDPMKVXDIRWNFEKFLVGPDGVPVMRWFXXTPV	180
protein_ancestor    EQKVFTFLKNSCPPTSDLLGSPKQLFWEPMKVHDIRWNFEKFLVGPDGVPVMRWFHRASV	179
protein_humansec    EQKVFTFLKNSXPPTSDLLGSSSQLFWEPMKVXDIRWNFEKFLVGPDGVPVMXWFXQAPV	180

protein_mousecys    RIVQSDIMEYLNQTS--	195
protein_ancestor    STVKSDILEYLKQFTPE	196
protein_humansec    STVKSDILEYLKQFNTX	197
"""


def extract_sequences_from_clustal(alignment_text):
    """Extract sequences from CLUSTAL alignment."""
    sequences = {}
    for line in alignment_text.strip().split('\n'):
        line = line.strip()
        if not line or line.startswith('CLUSTAL') or line.startswith('*') or line.startswith(':') or line.startswith('.'):
            continue
        parts = line.split()
        if len(parts) >= 2:
            seq_name = parts[0]
            seq_fragment = parts[1]
            if seq_name not in sequences:
                sequences[seq_name] = ""
            sequences[seq_name] += seq_fragment
    return sequences


def calculate_identity(seq1, seq2, ignore_gaps=True):
    """
    Calculate sequence identity (% identical residues).
    
    Parameters:
    -----------
    seq1, seq2 : str
        Sequences to compare
    ignore_gaps : bool
        If True, skip positions with gaps
    
    Returns:
    --------
    identity : float
        Percentage identity (0-100)
    matches : int
        Number of matching positions
    total : int
        Total positions compared
    """
    matches = 0
    total = 0
    
    for aa1, aa2 in zip(seq1, seq2):
        # Skip gaps if requested
        if ignore_gaps and (aa1 == '-' or aa2 == '-'):
            continue
        
        # Skip unknown residues (X, U, *)
        if aa1 in 'XU*' or aa2 in 'XU*':
            continue
        
        total += 1
        if aa1 == aa2:
            matches += 1
    
    if total == 0:
        return 0.0, 0, 0
    
    identity = (matches / total) * 100
    return identity, matches, total


def analyze_sequence_identity(sequences):
    """Analyze sequence identity to ancestor."""
    
    # Find ancestor sequence
    ancestor_key = None
    for key in sequences.keys():
        if 'ancestor' in key.lower():
            ancestor_key = key
            break
    
    if not ancestor_key:
        print("ERROR: No ancestor sequence found!")
        return None
    
    ancestor_seq = sequences[ancestor_key]
    
    # Calculate identity for all sequences vs ancestor
    results = {}
    
    for name, seq in sequences.items():
        if name == ancestor_key:
            continue
        
        identity, matches, total = calculate_identity(ancestor_seq, seq)
        
        # Get simple name
        if 'mouse' in name.lower():
            simple_name = 'Mouse'
        elif 'human' in name.lower():
            simple_name = 'Human'
        else:
            simple_name = name
        
        results[simple_name] = {
            'identity': identity,
            'matches': matches,
            'total': total,
            'full_name': name,
            'sequence': seq
        }
    
    return ancestor_seq, results


def visualize_identity_results(ancestor_seq, results, output_file="sequence_identity.png"):
    """Create visualization of sequence identity results."""
    
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 2, hspace=0.4, wspace=0.3)
    
    # Main title
    fig.suptitle('Sequence Identity Analysis: Distance from Ancestor', 
                fontsize=18, fontweight='bold', y=0.98)
    
    # 1. Bar chart of identity scores
    ax1 = fig.add_subplot(gs[0, :])
    
    names = list(results.keys())
    identities = [results[name]['identity'] for name in names]
    colors = ['#2ca02c' if 'Mouse' in name else '#d62728' for name in names]
    
    bars = ax1.bar(names, identities, color=colors, alpha=0.8, edgecolor='black', linewidth=2)
    ax1.axhline(y=100, color='black', linestyle='--', linewidth=1, alpha=0.5, label='100% Identity')
    
    # Add value labels on bars
    for bar, val in zip(bars, identities):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.1f}%',
                ha='center', va='bottom', fontsize=16, fontweight='bold')
    
    ax1.set_ylabel('Sequence Identity (%)', fontsize=14, fontweight='bold')
    ax1.set_title('% Identity to Ancestor Sequence', fontsize=16, fontweight='bold', pad=15)
    ax1.set_ylim(0, 110)
    ax1.grid(axis='y', alpha=0.3, linestyle='--')
    ax1.legend(fontsize=12)
    
    # 2. Detailed comparison table
    ax2 = fig.add_subplot(gs[1, :])
    ax2.axis('off')
    
    # Create table data
    table_data = [['Species', 'Identity (%)', 'Matches', 'Total Positions', 'Differences']]
    
    for name in names:
        r = results[name]
        diff = r['total'] - r['matches']
        table_data.append([
            name,
            f"{r['identity']:.2f}%",
            str(r['matches']),
            str(r['total']),
            str(diff)
        ])
    
    table = ax2.table(cellText=table_data, cellLoc='center', loc='center',
                     colWidths=[0.2, 0.2, 0.2, 0.2, 0.2])
    
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1, 2.5)
    
    # Style header row
    for i in range(5):
        cell = table[(0, i)]
        cell.set_facecolor('#1e3c72')
        cell.set_text_props(weight='bold', color='white', fontsize=13)
    
    # Color code data rows
    for i, name in enumerate(names, start=1):
        color = '#c3e6cb' if 'Mouse' in name else '#f5c6cb'
        for j in range(5):
            cell = table[(i, j)]
            cell.set_facecolor(color)
            if j == 1:  # Identity column
                cell.set_text_props(weight='bold', fontsize=13)
    
    ax2.set_title('Detailed Identity Comparison', fontsize=16, fontweight='bold', pad=20)
    
    # 3. Position-by-position differences
    ax3 = fig.add_subplot(gs[2, 0])
    
    # Count differences at each position
    mouse_seq = results['Mouse']['sequence']
    human_seq = results['Human']['sequence']
    
    position_diff = []
    positions = []
    
    for i, (anc, mouse, human) in enumerate(zip(ancestor_seq, mouse_seq, human_seq), 1):
        if anc == '-' or mouse == '-' or human == '-':
            continue
        if anc in 'XU*' or mouse in 'XU*' or human in 'XU*':
            continue
        
        positions.append(i)
        
        # Count how many sequences differ from ancestor at this position
        diff_count = 0
        if mouse != anc:
            diff_count += 1
        if human != anc:
            diff_count += 1
        
        position_diff.append(diff_count)
    
    # Plot
    colors_map = {0: 'green', 1: 'orange', 2: 'red'}
    bar_colors = [colors_map[d] for d in position_diff]
    
    ax3.bar(range(len(positions)), position_diff, color=bar_colors, alpha=0.7, edgecolor='black', linewidth=0.5)
    ax3.set_xlabel('Position in Alignment', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Number of Sequences\nDiffering from Ancestor', fontsize=12, fontweight='bold')
    ax3.set_title('Divergence at Each Position', fontsize=14, fontweight='bold', pad=15)
    ax3.set_yticks([0, 1, 2])
    ax3.set_yticklabels(['0\n(Conserved)', '1\n(1 differs)', '2\n(Both differ)'])
    ax3.grid(axis='y', alpha=0.3)
    
    # 4. Summary statistics
    ax4 = fig.add_subplot(gs[2, 1])
    ax4.axis('off')
    
    # Determine which is closer
    if results['Human']['identity'] > results['Mouse']['identity']:
        closer = 'Human'
        closer_val = results['Human']['identity']
        farther = 'Mouse'
        farther_val = results['Mouse']['identity']
    else:
        closer = 'Mouse'
        closer_val = results['Mouse']['identity']
        farther = 'Human'
        farther_val = results['Human']['identity']
    
    diff = abs(results['Human']['identity'] - results['Mouse']['identity'])
    
    summary_text = f"""
    SUMMARY
    {'='*40}
    
    CLOSEST TO ANCESTOR:
    {closer}: {closer_val:.2f}% identity
    
    MOST DIVERGED:
    {farther}: {farther_val:.2f}% identity
    
    DIFFERENCE:
    {diff:.2f} percentage points
    
    INTERPRETATION:
    {'='*40}
    
    {closer} GPX6 is MORE SIMILAR to the
    ancestral sequence, suggesting it has
    retained more ancestral characteristics.
    
    {farther} GPX6 has diverged MORE from
    the ancestor, indicating greater 
    evolutionary change.
    
    Both sequences show ~{100 - ((closer_val + farther_val)/2):.1f}%
    divergence on average, indicating
    significant evolution since their
    common ancestor.
    """
    
    ax4.text(0.05, 0.95, summary_text, transform=ax4.transAxes,
            fontsize=11, verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"✅ Identity visualization saved to: {output_file}")
    plt.close()


def create_text_report(ancestor_seq, results, output_file="sequence_identity_report.txt"):
    """Create detailed text report."""
    
    with open(output_file, 'w') as f:
        f.write("="*80 + "\n")
        f.write("SEQUENCE IDENTITY ANALYSIS REPORT\n")
        f.write("="*80 + "\n\n")
        
        f.write("ANALYSIS: Which sequence is closest to the ancestor?\n")
        f.write("-"*80 + "\n\n")
        
        # Results for each sequence
        for name, data in sorted(results.items(), key=lambda x: x[1]['identity'], reverse=True):
            f.write(f"{name}:\n")
            f.write(f"  Identity to ancestor:    {data['identity']:.2f}%\n")
            f.write(f"  Matching positions:      {data['matches']} / {data['total']}\n")
            f.write(f"  Different positions:     {data['total'] - data['matches']}\n")
            f.write(f"  Full sequence name:      {data['full_name']}\n")
            f.write("\n")
        
        # Determine closest
        sorted_results = sorted(results.items(), key=lambda x: x[1]['identity'], reverse=True)
        closest_name, closest_data = sorted_results[0]
        
        f.write("="*80 + "\n")
        f.write("CONCLUSION\n")
        f.write("="*80 + "\n\n")
        
        f.write(f"**{closest_name}** is CLOSEST to the ancestor sequence\n")
        f.write(f"with {closest_data['identity']:.2f}% identity ({closest_data['matches']}/{closest_data['total']} positions)\n\n")
        
        # Calculate difference
        if len(sorted_results) > 1:
            other_name, other_data = sorted_results[1]
            diff = closest_data['identity'] - other_data['identity']
            f.write(f"{closest_name} is {diff:.2f} percentage points closer to the ancestor than {other_name}.\n\n")
        
        # Evolutionary interpretation
        f.write("EVOLUTIONARY INTERPRETATION:\n")
        f.write("-"*80 + "\n\n")
        
        avg_identity = np.mean([r['identity'] for r in results.values()])
        avg_divergence = 100 - avg_identity
        
        f.write(f"Average sequence identity to ancestor: {avg_identity:.2f}%\n")
        f.write(f"Average divergence from ancestor:      {avg_divergence:.2f}%\n\n")
        
        f.write(f"Since the ancestor-descendant split (~90 million years ago for human-mouse):\n")
        f.write(f"  • {closest_name} has accumulated fewer changes (more conserved)\n")
        f.write(f"  • The high identity ({avg_identity:.1f}%) indicates strong functional constraints\n")
        f.write(f"  • Both lineages show adaptation, but {closest_name} retained more ancestral features\n\n")
        
        # Sequence comparison
        f.write("="*80 + "\n")
        f.write("POSITION-BY-POSITION DIFFERENCES\n")
        f.write("="*80 + "\n\n")
        
        f.write("Positions where sequences differ from ancestor:\n\n")
        
        mouse_seq = results['Mouse']['sequence']
        human_seq = results['Human']['sequence']
        
        f.write(f"{'Pos':<5} {'Anc':<5} {'Mouse':<7} {'Human':<7} {'Status'}\n")
        f.write("-"*50 + "\n")
        
        diff_count = 0
        for i, (anc, mouse, human) in enumerate(zip(ancestor_seq, mouse_seq, human_seq), 1):
            if anc == '-' or mouse == '-' or human == '-':
                continue
            if anc in 'XU*' or mouse in 'XU*' or human in 'XU*':
                continue
            
            if mouse != anc or human != anc:
                mouse_mark = "DIFF" if mouse != anc else "SAME"
                human_mark = "DIFF" if human != anc else "SAME"
                
                f.write(f"{i:<5} {anc:<5} {mouse:<7} {human:<7} M:{mouse_mark}, H:{human_mark}\n")
                diff_count += 1
                
                if diff_count > 50:  # Limit output
                    f.write(f"\n... ({diff_count - 50} more differences not shown) ...\n")
                    break
        
        f.write("\n" + "="*80 + "\n")
    
    print(f"✅ Text report saved to: {output_file}")


def main():
    """Main function."""
    
    print("="*80)
    print("SEQUENCE IDENTITY CALCULATOR")
    print("="*80)
    
    print("\n[1] Extracting sequences from alignment...")
    sequences = extract_sequences_from_clustal(CLUSTAL_ALIGNMENT)
    
    print(f"    ✓ Found {len(sequences)} sequences:")
    for name in sequences.keys():
        print(f"      - {name} ({len(sequences[name])} residues)")
    
    print("\n[2] Calculating sequence identity to ancestor...")
    ancestor_seq, results = analyze_sequence_identity(sequences)
    
    if results:
        print("\n    RESULTS:")
        print("    " + "-"*60)
        for name, data in sorted(results.items(), key=lambda x: x[1]['identity'], reverse=True):
            print(f"    {name:15} {data['identity']:6.2f}% identity ({data['matches']}/{data['total']} positions)")
        
        # Determine closest
        sorted_results = sorted(results.items(), key=lambda x: x[1]['identity'], reverse=True)
        closest = sorted_results[0][0]
        print(f"\n    ⭐ {closest} IS CLOSEST TO ANCESTOR!")
        
        print("\n[3] Creating visualizations...")
        visualize_identity_results(ancestor_seq, results)
        
        print("\n[4] Generating text report...")
        create_text_report(ancestor_seq, results)
    
    print("\n" + "="*80)
    print("✅ ANALYSIS COMPLETE")
    print("="*80)
    print("\nOutput files:")
    print("  • sequence_identity.png - Visual comparison of identity scores")
    print("  • sequence_identity_report.txt - Detailed text analysis")


if __name__ == "__main__":
    main()

SEQUENCE IDENTITY CALCULATOR

[1] Extracting sequences from alignment...
    ✓ Found 3 sequences:
      - protein_mousecys (197 residues)
      - protein_ancestor (197 residues)
      - protein_humansec (197 residues)

[2] Calculating sequence identity to ancestor...

    RESULTS:
    ------------------------------------------------------------
    Human            87.30% identity (165/189 positions)
    Mouse            78.61% identity (147/187 positions)

    ⭐ Human IS CLOSEST TO ANCESTOR!

[3] Creating visualizations...
✅ Identity visualization saved to: sequence_identity.png

[4] Generating text report...
✅ Text report saved to: sequence_identity_report.txt

✅ ANALYSIS COMPLETE

Output files:
  • sequence_identity.png - Visual comparison of identity scores
  • sequence_identity_report.txt - Detailed text analysis


In [8]:
#!/usr/bin/env python3
"""
Comprehensive Phylogenetic Analysis with BLOSUM Scores
Reads all level.fasta files and creates publication-quality phylogenetic tree
"""

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import seaborn as sns
from pathlib import Path
from collections import defaultdict
import re


# BLOSUM62 Matrix
BLOSUM62 = {
    'A': {'A': 4, 'R': -1, 'N': -2, 'D': -2, 'C': 0, 'Q': -1, 'E': -1, 'G': 0, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 0, 'W': -3, 'Y': -2, 'V': 0},
    'R': {'A': -1, 'R': 5, 'N': 0, 'D': -2, 'C': -3, 'Q': 1, 'E': 0, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 2, 'M': -1, 'F': -3, 'P': -2, 'S': -1, 'T': -1, 'W': -3, 'Y': -2, 'V': -3},
    'N': {'A': -2, 'R': 0, 'N': 6, 'D': 1, 'C': -3, 'Q': 0, 'E': 0, 'G': 0, 'H': 1, 'I': -3, 'L': -3, 'K': 0, 'M': -2, 'F': -3, 'P': -2, 'S': 1, 'T': 0, 'W': -4, 'Y': -2, 'V': -3},
    'D': {'A': -2, 'R': -2, 'N': 1, 'D': 6, 'C': -3, 'Q': 0, 'E': 2, 'G': -1, 'H': -1, 'I': -3, 'L': -4, 'K': -1, 'M': -3, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -4, 'Y': -3, 'V': -3},
    'C': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': 9, 'Q': -3, 'E': -4, 'G': -3, 'H': -3, 'I': -1, 'L': -1, 'K': -3, 'M': -1, 'F': -2, 'P': -3, 'S': -1, 'T': -1, 'W': -2, 'Y': -2, 'V': -1},
    'Q': {'A': -1, 'R': 1, 'N': 0, 'D': 0, 'C': -3, 'Q': 5, 'E': 2, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 1, 'M': 0, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -2, 'Y': -1, 'V': -2},
    'E': {'A': -1, 'R': 0, 'N': 0, 'D': 2, 'C': -4, 'Q': 2, 'E': 5, 'G': -2, 'H': 0, 'I': -3, 'L': -3, 'K': 1, 'M': -2, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -2},
    'G': {'A': 0, 'R': -2, 'N': 0, 'D': -1, 'C': -3, 'Q': -2, 'E': -2, 'G': 6, 'H': -2, 'I': -4, 'L': -4, 'K': -2, 'M': -3, 'F': -3, 'P': -2, 'S': 0, 'T': -2, 'W': -2, 'Y': -3, 'V': -3},
    'H': {'A': -2, 'R': 0, 'N': 1, 'D': -1, 'C': -3, 'Q': 0, 'E': 0, 'G': -2, 'H': 8, 'I': -3, 'L': -3, 'K': -1, 'M': -2, 'F': -1, 'P': -2, 'S': -1, 'T': -2, 'W': -2, 'Y': 2, 'V': -3},
    'I': {'A': -1, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -3, 'E': -3, 'G': -4, 'H': -3, 'I': 4, 'L': 2, 'K': -3, 'M': 1, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -3, 'Y': -1, 'V': 3},
    'L': {'A': -1, 'R': -2, 'N': -3, 'D': -4, 'C': -1, 'Q': -2, 'E': -3, 'G': -4, 'H': -3, 'I': 2, 'L': 4, 'K': -2, 'M': 2, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -2, 'Y': -1, 'V': 1},
    'K': {'A': -1, 'R': 2, 'N': 0, 'D': -1, 'C': -3, 'Q': 1, 'E': 1, 'G': -2, 'H': -1, 'I': -3, 'L': -2, 'K': 5, 'M': -1, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -2},
    'M': {'A': -1, 'R': -1, 'N': -2, 'D': -3, 'C': -1, 'Q': 0, 'E': -2, 'G': -3, 'H': -2, 'I': 1, 'L': 2, 'K': -1, 'M': 5, 'F': 0, 'P': -2, 'S': -1, 'T': -1, 'W': -1, 'Y': -1, 'V': 1},
    'F': {'A': -2, 'R': -3, 'N': -3, 'D': -3, 'C': -2, 'Q': -3, 'E': -3, 'G': -3, 'H': -1, 'I': 0, 'L': 0, 'K': -3, 'M': 0, 'F': 6, 'P': -4, 'S': -2, 'T': -2, 'W': 1, 'Y': 3, 'V': -1},
    'P': {'A': -1, 'R': -2, 'N': -2, 'D': -1, 'C': -3, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -3, 'L': -3, 'K': -1, 'M': -2, 'F': -4, 'P': 7, 'S': -1, 'T': -1, 'W': -4, 'Y': -3, 'V': -2},
    'S': {'A': 1, 'R': -1, 'N': 1, 'D': 0, 'C': -1, 'Q': 0, 'E': 0, 'G': 0, 'H': -1, 'I': -2, 'L': -2, 'K': 0, 'M': -1, 'F': -2, 'P': -1, 'S': 4, 'T': 1, 'W': -3, 'Y': -2, 'V': -2},
    'T': {'A': 0, 'R': -1, 'N': 0, 'D': -1, 'C': -1, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 5, 'W': -2, 'Y': -2, 'V': 0},
    'W': {'A': -3, 'R': -3, 'N': -4, 'D': -4, 'C': -2, 'Q': -2, 'E': -3, 'G': -2, 'H': -2, 'I': -3, 'L': -2, 'K': -3, 'M': -1, 'F': 1, 'P': -4, 'S': -3, 'T': -2, 'W': 11, 'Y': 2, 'V': -3},
    'Y': {'A': -2, 'R': -2, 'N': -2, 'D': -3, 'C': -2, 'Q': -1, 'E': -2, 'G': -3, 'H': 2, 'I': -1, 'L': -1, 'K': -2, 'M': -1, 'F': 3, 'P': -3, 'S': -2, 'T': -2, 'W': 2, 'Y': 7, 'V': -1},
    'V': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -2, 'E': -2, 'G': -3, 'H': -3, 'I': 3, 'L': 1, 'K': -2, 'M': 1, 'F': -1, 'P': -2, 'S': -2, 'T': 0, 'W': -3, 'Y': -1, 'V': 4}
}

# Core sequences from CLUSTAL alignment
CORE_SEQUENCES = {
    'Ancestor': 'SQKMKMDCYKGVTGTIYEYGALTLNGEEYIQFKQYAGKHVLFINVATYGLTAQYPELNALQEELKPFGVVLLGFPCNQFGKQEPGKNSEILSGLKYVRPGGGFVPNFQLFEKGDVNGEKEQKVFTFLKNSCPPTSDLLGSPKQLFWEPMKVHDIRWNFEKFLVGPDGVPVMRWFHRASVSTVKSDILEYLKQFTPE',
    'Mouse_WT': 'PQKSKVDXNKGVTGTVYEYGANTIDGGEFVNFQQYAGKXILFVNVASFCGLTATYPELNTLQEELKPFNVTVLGFPCNQFGKQEPGKNSEILLGLKYVRPGGGYVPNFQLFEKGDVNGDNEQKVFSFLKNSXPPTSELFGSPEXLFWDPMKVXDIRWNFEKFLVGPDGVPVMRWFXXTPVRIVQSDIMEYLNQTS',
    'Human_WT': 'PQNRKVDXNKGVTGTIYEYGALTLNGEEYIQFKQFAGKXVLFVNVAAYUGLAAQYPELNALQEELKNFGVIVLAFPCNQFGKQEPGTNSEILLGLKYVCPGSGFVPSFQLFEKGDVNGEKEQKVFTFLKNSXPPTSDLLGSSSQLFWEPMKVXDIRWNFEKFLVGPDGVPVMXWFXQAPVSTVKSDILEYLKQFNTX'
}


def read_fasta_file(filepath):
    """Read a FASTA file and return sequences."""
    sequences = {}
    current_name = None
    current_seq = []
    
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                
                if line.startswith('>'):
                    # Save previous sequence
                    if current_name:
                        sequences[current_name] = ''.join(current_seq)
                    
                    # Start new sequence
                    current_name = line[1:].strip()
                    current_seq = []
                else:
                    current_seq.append(line.upper())
            
            # Save last sequence
            if current_name:
                sequences[current_name] = ''.join(current_seq)
    
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return {}
    
    return sequences


def read_all_level_files(mouse_dir, human_dir):
    """Read all level*.fasta files from both directories."""
    all_sequences = {}
    
    # Read mouse files
    mouse_path = Path(mouse_dir)
    if mouse_path.exists():
        # Try both naming patterns
        fasta_files = list(mouse_path.glob('GPX6_level*.fasta')) + list(mouse_path.glob('level*.fasta'))
        for fasta_file in sorted(set(fasta_files)):
            level = re.search(r'level(\d+)', fasta_file.name)
            if level:
                level_num = int(level.group(1))
                seqs = read_fasta_file(fasta_file)
                for name, seq in seqs.items():
                    # Store with pathway information
                    key = f"M2H_Level{level_num:02d}"
                    all_sequences[key] = {
                        'sequence': seq,
                        'level': level_num,
                        'pathway': 'Mouse_to_Human',
                        'original_name': name
                    }
                print(f"  ✓ Read {fasta_file.name}: {len(seqs)} sequence(s)")
    
    # Read human files
    human_path = Path(human_dir)
    if human_path.exists():
        # Try both naming patterns
        fasta_files = list(human_path.glob('GPX6_level*.fasta')) + list(human_path.glob('level*.fasta'))
        for fasta_file in sorted(set(fasta_files)):
            level = re.search(r'level(\d+)', fasta_file.name)
            if level:
                level_num = int(level.group(1))
                seqs = read_fasta_file(fasta_file)
                for name, seq in seqs.items():
                    # Store with pathway information
                    key = f"H2M_Level{level_num:02d}"
                    all_sequences[key] = {
                        'sequence': seq,
                        'level': level_num,
                        'pathway': 'Human_to_Mouse',
                        'original_name': name
                    }
                print(f"  ✓ Read {fasta_file.name}: {len(seqs)} sequence(s)")
    
    return all_sequences


def calculate_identity(seq1, seq2):
    """Calculate sequence identity percentage."""
    matches = 0
    total = 0
    
    for aa1, aa2 in zip(seq1, seq2):
        if aa1 in '-XU*' or aa2 in '-XU*':
            continue
        total += 1
        if aa1 == aa2:
            matches += 1
    
    return (matches / total * 100) if total > 0 else 0.0


def calculate_blosum_score(seq1, seq2, normalized=True):
    """Calculate BLOSUM62 score between two sequences."""
    score = 0
    comparisons = 0
    
    for aa1, aa2 in zip(seq1, seq2):
        if aa1 in '-XU*' or aa2 in '-XU*':
            continue
        
        if aa1 in BLOSUM62 and aa2 in BLOSUM62[aa1]:
            score += BLOSUM62[aa1][aa2]
            comparisons += 1
    
    if normalized and comparisons > 0:
        return score / comparisons
    return score


def create_similarity_matrices(sequences_dict):
    """Create identity and BLOSUM score matrices for all sequences."""
    
    # Combine core sequences with level sequences
    all_seqs = CORE_SEQUENCES.copy()
    for key, data in sequences_dict.items():
        all_seqs[key] = data['sequence']
    
    seq_names = list(all_seqs.keys())
    n = len(seq_names)
    
    identity_matrix = np.zeros((n, n))
    blosum_matrix = np.zeros((n, n))
    
    for i, name1 in enumerate(seq_names):
        for j, name2 in enumerate(seq_names):
            seq1 = all_seqs[name1]
            seq2 = all_seqs[name2]
            
            identity_matrix[i, j] = calculate_identity(seq1, seq2)
            blosum_matrix[i, j] = calculate_blosum_score(seq1, seq2)
    
    return identity_matrix, blosum_matrix, seq_names


def visualize_similarity_heatmaps(identity_matrix, blosum_matrix, seq_names, 
                                  output_file="similarity_heatmaps.png"):
    """Create heatmaps for identity and BLOSUM scores."""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    
    # Identity heatmap
    im1 = ax1.imshow(identity_matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=100)
    ax1.set_xticks(range(len(seq_names)))
    ax1.set_yticks(range(len(seq_names)))
    ax1.set_xticklabels(seq_names, rotation=90, ha='right', fontsize=8)
    ax1.set_yticklabels(seq_names, fontsize=8)
    ax1.set_title('Sequence Identity Matrix (%)', fontsize=16, fontweight='bold', pad=20)
    
    cbar1 = plt.colorbar(im1, ax=ax1)
    cbar1.set_label('Identity (%)', fontsize=12, rotation=270, labelpad=20)
    
    # BLOSUM heatmap
    im2 = ax2.imshow(blosum_matrix, cmap='RdBu_r', aspect='auto')
    ax2.set_xticks(range(len(seq_names)))
    ax2.set_yticks(range(len(seq_names)))
    ax2.set_xticklabels(seq_names, rotation=90, ha='right', fontsize=8)
    ax2.set_yticklabels(seq_names, fontsize=8)
    ax2.set_title('BLOSUM62 Similarity Matrix', fontsize=16, fontweight='bold', pad=20)
    
    cbar2 = plt.colorbar(im2, ax=ax2)
    cbar2.set_label('BLOSUM Score (normalized)', fontsize=12, rotation=270, labelpad=20)
    
    plt.suptitle('Comprehensive Sequence Similarity Analysis', 
                fontsize=18, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"✅ Similarity heatmaps saved to: {output_file}")
    plt.close()


def create_publication_phylogenetic_tree(identity_matrix, seq_names, 
                                        output_file="publication_phylogenetic_tree.png"):
    """Create publication-quality phylogenetic tree."""
    
    fig, ax = plt.subplots(figsize=(18, 14))
    ax.set_xlim(0, 12)
    ax.set_ylim(0, 12)
    ax.axis('off')
    
    # Colors
    ancestor_color = '#FFD700'  # Gold
    human_color = '#d62728'     # Red
    mouse_color = '#2ca02c'     # Green
    h2m_color = '#ff7f0e'       # Orange
    m2h_color = '#9467bd'       # Purple
    
    # Root and ancestor positions
    root_x, root_y = 1, 6
    ancestor_x, ancestor_y = 3, 6
    
    # Main branch splits
    human_split_x, human_split_y = 5, 8
    mouse_split_x, mouse_split_y = 5, 4
    
    # Terminal positions
    human_wt_x, human_wt_y = 11, 9.5
    mouse_wt_x, mouse_wt_y = 11, 2.5
    
    # Draw main branches
    branches = [
        # Root to ancestor
        (root_x, root_y, ancestor_x, ancestor_y, 'black', 3, '-'),
        # Ancestor to splits
        (ancestor_x, ancestor_y, human_split_x, human_split_y, human_color, 2.5, '-'),
        (ancestor_x, ancestor_y, mouse_split_x, mouse_split_y, mouse_color, 2.5, '--'),
        # To terminals
        (human_split_x, human_split_y, human_wt_x, human_wt_y, human_color, 2.5, '-'),
        (mouse_split_x, mouse_split_y, mouse_wt_x, mouse_wt_y, mouse_color, 2.5, '--'),
    ]
    
    for x1, y1, x2, y2, color, lw, style in branches:
        ax.plot([x1, x2], [y1, y2], color=color, linewidth=lw, 
               linestyle=style, alpha=0.8, zorder=1)
    
    # Draw mutation level branches (simplified representation)
    # Human to Mouse pathway
    h2m_y_positions = np.linspace(human_wt_y - 0.5, mouse_wt_y + 0.5, 5)
    for i, y in enumerate(h2m_y_positions):
        level_x = human_wt_x - 1.5
        ax.plot([human_wt_x - 0.5, level_x], [human_wt_y, y], 
               color=h2m_color, linewidth=1, alpha=0.3, linestyle=':')
        ax.scatter(level_x, y, s=30, c=h2m_color, alpha=0.6, zorder=2)
    
    # Mouse to Human pathway
    m2h_y_positions = np.linspace(mouse_wt_y + 0.5, human_wt_y - 0.5, 5)
    for i, y in enumerate(m2h_y_positions):
        level_x = mouse_wt_x - 1.5
        ax.plot([mouse_wt_x - 0.5, level_x], [mouse_wt_y, y], 
               color=m2h_color, linewidth=1, alpha=0.3, linestyle=':')
        ax.scatter(level_x, y, s=30, c=m2h_color, alpha=0.6, zorder=2)
    
    # Draw main nodes
    nodes = [
        (root_x, root_y, 'Root', 200, 'black', 12),
        (ancestor_x, ancestor_y, 'Ancestor\nNode 23', 400, ancestor_color, 14),
        (human_split_x, human_split_y, '', 150, human_color, 10),
        (mouse_split_x, mouse_split_y, '', 150, mouse_color, 10),
    ]
    
    for x, y, label, size, color, fontsize in nodes:
        ax.scatter(x, y, s=size, c=color, zorder=5, 
                  edgecolors='black', linewidths=2)
        if label:
            bbox_props = dict(boxstyle='round,pad=0.6', facecolor='white', 
                            edgecolor='black', linewidth=2, alpha=0.95)
            ax.text(x, y - 0.5, label, ha='center', va='top',
                   fontsize=fontsize, fontweight='bold', bbox=bbox_props)
    
    # Draw terminal nodes with boxes
    terminals = [
        (human_wt_x, human_wt_y, 'Human WT\n(Sec)', human_color, 
         '87.3% identity\nto ancestor'),
        (mouse_wt_x, mouse_wt_y, 'Mouse WT\n(Cys)', mouse_color,
         '78.6% identity\nto ancestor'),
    ]
    
    for x, y, label, color, info in terminals:
        # Main node
        ax.scatter(x, y, s=400, c=color, zorder=5,
                  edgecolors='black', linewidths=3, marker='s')
        
        # Label box
        bbox_props = dict(boxstyle='round,pad=0.8', facecolor='white',
                        edgecolor=color, linewidth=3, alpha=0.95)
        text_str = f"{label}\n\n{info}"
        ax.text(x + 0.5, y, text_str, ha='left', va='center',
               fontsize=11, fontweight='bold', bbox=bbox_props)
    
    # Add pathway labels
    ax.text(human_wt_x - 2, (human_wt_y + mouse_wt_y)/2 + 1.5,
           'H→M Pathway\n(Mutation Levels)',
           ha='center', va='center', fontsize=10, style='italic',
           color=h2m_color, fontweight='bold',
           bbox=dict(boxstyle='round,pad=0.5', facecolor='white', 
                    edgecolor=h2m_color, linewidth=2, alpha=0.8))
    
    ax.text(mouse_wt_x - 2, (mouse_wt_y + human_wt_y)/2 - 1.5,
           'M→H Pathway\n(Mutation Levels)',
           ha='center', va='center', fontsize=10, style='italic',
           color=m2h_color, fontweight='bold',
           bbox=dict(boxstyle='round,pad=0.5', facecolor='white',
                    edgecolor=m2h_color, linewidth=2, alpha=0.8))
    
    # Title
    ax.text(6, 11.5, 'Phylogenetic Tree: GPX6 Evolution and Mutation Pathways',
           ha='center', va='top', fontsize=20, fontweight='bold')
    
    # Legend
    legend_elements = [
        Line2D([0], [0], color=human_color, linewidth=3, linestyle='-',
              label='Sec-containing lineage (Human)'),
        Line2D([0], [0], color=mouse_color, linewidth=3, linestyle='--',
              label='Cys-containing lineage (Mouse)'),
        Line2D([0], [0], color=h2m_color, linewidth=2, linestyle=':',
              label='Human→Mouse mutation pathway', alpha=0.6),
        Line2D([0], [0], color=m2h_color, linewidth=2, linestyle=':',
              label='Mouse→Human mutation pathway', alpha=0.6),
        Line2D([0], [0], marker='o', color='w', markerfacecolor=ancestor_color,
              markersize=15, markeredgecolor='black', markeredgewidth=2,
              label='Ancestral node'),
        Line2D([0], [0], marker='s', color='w', markerfacecolor='gray',
              markersize=12, markeredgecolor='black', markeredgewidth=2,
              label='Wild-type sequences'),
    ]
    
    ax.legend(handles=legend_elements, loc='lower left', fontsize=11,
             framealpha=0.95, edgecolor='black', fancybox=True)
    
    # Scale bar
    ax.plot([1, 2.5], [0.8, 0.8], 'k-', linewidth=3)
    ax.text(1.75, 0.5, 'Evolutionary distance', ha='center', fontsize=10,
           style='italic')
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"✅ Publication phylogenetic tree saved to: {output_file}")
    plt.close()


def create_identity_summary_table(identity_matrix, seq_names, 
                                  output_file="identity_summary.txt"):
    """Create summary table of identity scores."""
    
    # Find indices of core sequences
    ancestor_idx = seq_names.index('Ancestor') if 'Ancestor' in seq_names else None
    human_idx = seq_names.index('Human_WT') if 'Human_WT' in seq_names else None
    mouse_idx = seq_names.index('Mouse_WT') if 'Mouse_WT' in seq_names else None
    
    with open(output_file, 'w') as f:
        f.write("="*80 + "\n")
        f.write("COMPREHENSIVE SEQUENCE IDENTITY SUMMARY\n")
        f.write("="*80 + "\n\n")
        
        f.write("CORE SEQUENCES IDENTITY TO ANCESTOR:\n")
        f.write("-"*80 + "\n")
        
        if ancestor_idx is not None:
            if human_idx is not None:
                human_id = identity_matrix[human_idx, ancestor_idx]
                f.write(f"Human WT:     {human_id:.2f}%\n")
            
            if mouse_idx is not None:
                mouse_id = identity_matrix[mouse_idx, ancestor_idx]
                f.write(f"Mouse WT:     {mouse_id:.2f}%\n")
            
            f.write("\n")
        
        # Mutation levels identity to ancestor
        f.write("MUTATION LEVEL SEQUENCES IDENTITY TO ANCESTOR:\n")
        f.write("-"*80 + "\n\n")
        
        if ancestor_idx is not None:
            # Human to Mouse pathway
            f.write("Human → Mouse Pathway:\n")
            h2m_levels = [(name, i) for i, name in enumerate(seq_names) if 'H2M_' in name]
            for name, idx in sorted(h2m_levels):
                identity = identity_matrix[idx, ancestor_idx]
                f.write(f"  {name:20} {identity:6.2f}%\n")
            
            f.write("\n")
            
            # Mouse to Human pathway
            f.write("Mouse → Human Pathway:\n")
            m2h_levels = [(name, i) for i, name in enumerate(seq_names) if 'M2H_' in name]
            for name, idx in sorted(m2h_levels):
                identity = identity_matrix[idx, ancestor_idx]
                f.write(f"  {name:20} {identity:6.2f}%\n")
        
        f.write("\n" + "="*80 + "\n")
        
        # BLOSUM scores section
        f.write("\nBLOSUM62 SCORES TO ANCESTOR:\n")
        f.write("-"*80 + "\n")
        
        if ancestor_idx is not None:
            if human_idx is not None:
                # Calculate BLOSUM for Human
                human_seq = CORE_SEQUENCES['Human_WT']
                anc_seq = CORE_SEQUENCES['Ancestor']
                human_blosum = calculate_blosum_score(anc_seq, human_seq)
                f.write(f"Human WT:     {human_blosum:.3f} (normalized)\n")
            
            if mouse_idx is not None:
                # Calculate BLOSUM for Mouse
                mouse_seq = CORE_SEQUENCES['Mouse_WT']
                anc_seq = CORE_SEQUENCES['Ancestor']
                mouse_blosum = calculate_blosum_score(anc_seq, mouse_seq)
                f.write(f"Mouse WT:     {mouse_blosum:.3f} (normalized)\n")
        
        f.write("\n" + "="*80 + "\n")
    
    print(f"✅ Identity summary table saved to: {output_file}")


def main():
    """Main analysis function."""
    
    print("="*80)
    print("COMPREHENSIVE PHYLOGENETIC ANALYSIS WITH BLOSUM SCORES")
    print("="*80)
    
    # Directories - YOUR ACTUAL PATHS
    mouse_dir = "/home/hp/nayanika/github/GPX6/analysis/alignment/MOUSE"
    human_dir = "/home/hp/nayanika/github/GPX6/analysis/alignment/HUMAN"
    
    # Alternative if you want to use current directory:
    # mouse_dir = "./MOUSE"
    # human_dir = "./HUMAN"
    
    print(f"\n[1] Reading FASTA files from:")
    print(f"    Mouse: {mouse_dir}")
    print(f"    Human: {human_dir}")
    
    level_sequences = read_all_level_files(mouse_dir, human_dir)
    
    if not level_sequences:
        print("\n⚠️  WARNING: No level files found!")
        print("    Using only core sequences (Ancestor, Human_WT, Mouse_WT)")
        print("    To include mutation levels, ensure:")
        print("    1. FASTA files are named 'level*.fasta' or 'GPX6_level*.fasta'")
        print("    2. They are in the correct directories")
        print("    3. Paths are correctly specified\n")
    else:
        print(f"\n    ✓ Loaded {len(level_sequences)} mutation level sequences")
    
    print("\n[2] Calculating similarity matrices...")
    identity_matrix, blosum_matrix, seq_names = create_similarity_matrices(level_sequences)
    print(f"    ✓ Calculated {len(seq_names)}x{len(seq_names)} matrices")
    
    print("\n[3] Creating visualizations...")
    visualize_similarity_heatmaps(identity_matrix, blosum_matrix, seq_names)
    
    print("\n[4] Creating publication-quality phylogenetic tree...")
    create_publication_phylogenetic_tree(identity_matrix, seq_names)
    
    print("\n[5] Generating identity summary...")
    create_identity_summary_table(identity_matrix, seq_names)
    
    print("\n" + "="*80)
    print("✅ ANALYSIS COMPLETE")
    print("="*80)
    print("\nOutput files:")
    print("  • similarity_heatmaps.png - Identity & BLOSUM score matrices")
    print("  • publication_phylogenetic_tree.png - Publication-quality tree")
    print("  • identity_summary.txt - Detailed identity scores")
    print("\nFor publication:")
    print("  - Use publication_phylogenetic_tree.png as Figure")
    print("  - Include identity_summary.txt data in supplementary materials")
    print("  - Heatmaps can support main findings")


if __name__ == "__main__":
    main()

COMPREHENSIVE PHYLOGENETIC ANALYSIS WITH BLOSUM SCORES

[1] Reading FASTA files from:
    Mouse: /home/hp/nayanika/github/GPX6/analysis/alignment/MOUSE
    Human: /home/hp/nayanika/github/GPX6/analysis/alignment/HUMAN
  ✓ Read GPX6_level01.fasta: 1 sequence(s)
  ✓ Read GPX6_level02.fasta: 1 sequence(s)
  ✓ Read GPX6_level03.fasta: 1 sequence(s)
  ✓ Read GPX6_level04.fasta: 1 sequence(s)
  ✓ Read GPX6_level05.fasta: 1 sequence(s)
  ✓ Read GPX6_level06.fasta: 1 sequence(s)
  ✓ Read GPX6_level07.fasta: 1 sequence(s)
  ✓ Read GPX6_level08.fasta: 1 sequence(s)
  ✓ Read GPX6_level09.fasta: 1 sequence(s)
  ✓ Read GPX6_level10.fasta: 1 sequence(s)
  ✓ Read GPX6_level11.fasta: 1 sequence(s)
  ✓ Read GPX6_level12.fasta: 1 sequence(s)
  ✓ Read GPX6_level13.fasta: 1 sequence(s)
  ✓ Read GPX6_level14.fasta: 1 sequence(s)
  ✓ Read GPX6_level15.fasta: 1 sequence(s)
  ✓ Read GPX6_level16.fasta: 1 sequence(s)
  ✓ Read GPX6_level17.fasta: 1 sequence(s)
  ✓ Read GPX6_level18.fasta: 1 sequence(s)
  ✓ Read

In [9]:
#!/usr/bin/env python3
"""
Publication-Quality Phylogenetic Tree from Similarity Matrix
Uses hierarchical clustering to create accurate evolutionary tree
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
import seaborn as sns
from pathlib import Path
import re


# BLOSUM62 Matrix
BLOSUM62 = {
    'A': {'A': 4, 'R': -1, 'N': -2, 'D': -2, 'C': 0, 'Q': -1, 'E': -1, 'G': 0, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 0, 'W': -3, 'Y': -2, 'V': 0},
    'R': {'A': -1, 'R': 5, 'N': 0, 'D': -2, 'C': -3, 'Q': 1, 'E': 0, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 2, 'M': -1, 'F': -3, 'P': -2, 'S': -1, 'T': -1, 'W': -3, 'Y': -2, 'V': -3},
    'N': {'A': -2, 'R': 0, 'N': 6, 'D': 1, 'C': -3, 'Q': 0, 'E': 0, 'G': 0, 'H': 1, 'I': -3, 'L': -3, 'K': 0, 'M': -2, 'F': -3, 'P': -2, 'S': 1, 'T': 0, 'W': -4, 'Y': -2, 'V': -3},
    'D': {'A': -2, 'R': -2, 'N': 1, 'D': 6, 'C': -3, 'Q': 0, 'E': 2, 'G': -1, 'H': -1, 'I': -3, 'L': -4, 'K': -1, 'M': -3, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -4, 'Y': -3, 'V': -3},
    'C': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': 9, 'Q': -3, 'E': -4, 'G': -3, 'H': -3, 'I': -1, 'L': -1, 'K': -3, 'M': -1, 'F': -2, 'P': -3, 'S': -1, 'T': -1, 'W': -2, 'Y': -2, 'V': -1},
    'Q': {'A': -1, 'R': 1, 'N': 0, 'D': 0, 'C': -3, 'Q': 5, 'E': 2, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 1, 'M': 0, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -2, 'Y': -1, 'V': -2},
    'E': {'A': -1, 'R': 0, 'N': 0, 'D': 2, 'C': -4, 'Q': 2, 'E': 5, 'G': -2, 'H': 0, 'I': -3, 'L': -3, 'K': 1, 'M': -2, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -2},
    'G': {'A': 0, 'R': -2, 'N': 0, 'D': -1, 'C': -3, 'Q': -2, 'E': -2, 'G': 6, 'H': -2, 'I': -4, 'L': -4, 'K': -2, 'M': -3, 'F': -3, 'P': -2, 'S': 0, 'T': -2, 'W': -2, 'Y': -3, 'V': -3},
    'H': {'A': -2, 'R': 0, 'N': 1, 'D': -1, 'C': -3, 'Q': 0, 'E': 0, 'G': -2, 'H': 8, 'I': -3, 'L': -3, 'K': -1, 'M': -2, 'F': -1, 'P': -2, 'S': -1, 'T': -2, 'W': -2, 'Y': 2, 'V': -3},
    'I': {'A': -1, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -3, 'E': -3, 'G': -4, 'H': -3, 'I': 4, 'L': 2, 'K': -3, 'M': 1, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -3, 'Y': -1, 'V': 3},
    'L': {'A': -1, 'R': -2, 'N': -3, 'D': -4, 'C': -1, 'Q': -2, 'E': -3, 'G': -4, 'H': -3, 'I': 2, 'L': 4, 'K': -2, 'M': 2, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -2, 'Y': -1, 'V': 1},
    'K': {'A': -1, 'R': 2, 'N': 0, 'D': -1, 'C': -3, 'Q': 1, 'E': 1, 'G': -2, 'H': -1, 'I': -3, 'L': -2, 'K': 5, 'M': -1, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -2},
    'M': {'A': -1, 'R': -1, 'N': -2, 'D': -3, 'C': -1, 'Q': 0, 'E': -2, 'G': -3, 'H': -2, 'I': 1, 'L': 2, 'K': -1, 'M': 5, 'F': 0, 'P': -2, 'S': -1, 'T': -1, 'W': -1, 'Y': -1, 'V': 1},
    'F': {'A': -2, 'R': -3, 'N': -3, 'D': -3, 'C': -2, 'Q': -3, 'E': -3, 'G': -3, 'H': -1, 'I': 0, 'L': 0, 'K': -3, 'M': 0, 'F': 6, 'P': -4, 'S': -2, 'T': -2, 'W': 1, 'Y': 3, 'V': -1},
    'P': {'A': -1, 'R': -2, 'N': -2, 'D': -1, 'C': -3, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -3, 'L': -3, 'K': -1, 'M': -2, 'F': -4, 'P': 7, 'S': -1, 'T': -1, 'W': -4, 'Y': -3, 'V': -2},
    'S': {'A': 1, 'R': -1, 'N': 1, 'D': 0, 'C': -1, 'Q': 0, 'E': 0, 'G': 0, 'H': -1, 'I': -2, 'L': -2, 'K': 0, 'M': -1, 'F': -2, 'P': -1, 'S': 4, 'T': 1, 'W': -3, 'Y': -2, 'V': -2},
    'T': {'A': 0, 'R': -1, 'N': 0, 'D': -1, 'C': -1, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 5, 'W': -2, 'Y': -2, 'V': 0},
    'W': {'A': -3, 'R': -3, 'N': -4, 'D': -4, 'C': -2, 'Q': -2, 'E': -3, 'G': -2, 'H': -2, 'I': -3, 'L': -2, 'K': -3, 'M': -1, 'F': 1, 'P': -4, 'S': -3, 'T': -2, 'W': 11, 'Y': 2, 'V': -3},
    'Y': {'A': -2, 'R': -2, 'N': -2, 'D': -3, 'C': -2, 'Q': -1, 'E': -2, 'G': -3, 'H': 2, 'I': -1, 'L': -1, 'K': -2, 'M': -1, 'F': 3, 'P': -3, 'S': -2, 'T': -2, 'W': 2, 'Y': 7, 'V': -1},
    'V': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -2, 'E': -2, 'G': -3, 'H': -3, 'I': 3, 'L': 1, 'K': -2, 'M': 1, 'F': -1, 'P': -2, 'S': -2, 'T': 0, 'W': -3, 'Y': -1, 'V': 4}
}


# Core sequences
CORE_SEQUENCES = {
    'Ancestor': 'SQKMKMDCYKGVTGTIYEYGALTLNGEEYIQFKQYAGKHVLFINVATYGLTAQYPELNALQEELKPFGVVLLGFPCNQFGKQEPGKNSEILSGLKYVRPGGGFVPNFQLFEKGDVNGEKEQKVFTFLKNSCPPTSDLLGSPKQLFWEPMKVHDIRWNFEKFLVGPDGVPVMRWFHRASVSTVKSDILEYLKQFTPE',
    'Mouse_WT': 'PQKSKVDXNKGVTGTVYEYGANTIDGGEFVNFQQYAGKXILFVNVASFCGLTATYPELNTLQEELKPFNVTVLGFPCNQFGKQEPGKNSEILLGLKYVRPGGGYVPNFQLFEKGDVNGDNEQKVFSFLKNSXPPTSELFGSPEXLFWDPMKVXDIRWNFEKFLVGPDGVPVMRWFXXTPVRIVQSDIMEYLNQTS',
    'Human_WT': 'PQNRKVDXNKGVTGTIYEYGALTLNGEEYIQFKQFAGKXVLFVNVAAYUGLAAQYPELNALQEELKNFGVIVLAFPCNQFGKQEPGTNSEILLGLKYVCPGSGFVPSFQLFEKGDVNGEKEQKVFTFLKNSXPPTSDLLGSSSQLFWEPMKVXDIRWNFEKFLVGPDGVPVMXWFXQAPVSTVKSDILEYLKQFNTX'
}


def read_fasta_file(filepath):
    """Read a FASTA file and return sequences."""
    sequences = {}
    current_name = None
    current_seq = []
    
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                
                if line.startswith('>'):
                    if current_name:
                        sequences[current_name] = ''.join(current_seq)
                    current_name = line[1:].strip()
                    current_seq = []
                else:
                    current_seq.append(line.upper())
            
            if current_name:
                sequences[current_name] = ''.join(current_seq)
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return {}
    
    return sequences


def read_all_level_files(mouse_dir, human_dir):
    """Read all level files from both directories."""
    all_sequences = {}
    
    mouse_path = Path(mouse_dir)
    if mouse_path.exists():
        fasta_files = list(mouse_path.glob('GPX6_level*.fasta')) + list(mouse_path.glob('level*.fasta'))
        for fasta_file in sorted(set(fasta_files)):
            level = re.search(r'level(\d+)', fasta_file.name)
            if level:
                level_num = int(level.group(1))
                seqs = read_fasta_file(fasta_file)
                for name, seq in seqs.items():
                    key = f"M2H_Level{level_num:02d}"
                    all_sequences[key] = seq
                print(f"  ✓ Read {fasta_file.name}")
    
    human_path = Path(human_dir)
    if human_path.exists():
        fasta_files = list(human_path.glob('GPX6_level*.fasta')) + list(human_path.glob('level*.fasta'))
        for fasta_file in sorted(set(fasta_files)):
            level = re.search(r'level(\d+)', fasta_file.name)
            if level:
                level_num = int(level.group(1))
                seqs = read_fasta_file(fasta_file)
                for name, seq in seqs.items():
                    key = f"H2M_Level{level_num:02d}"
                    all_sequences[key] = seq
                print(f"  ✓ Read {fasta_file.name}")
    
    return all_sequences


def calculate_identity(seq1, seq2):
    """Calculate sequence identity."""
    matches = 0
    total = 0
    
    for aa1, aa2 in zip(seq1, seq2):
        if aa1 in '-XU*' or aa2 in '-XU*':
            continue
        total += 1
        if aa1 == aa2:
            matches += 1
    
    return (matches / total * 100) if total > 0 else 0.0


def create_distance_matrix(sequences_dict):
    """Create distance matrix from sequence identities."""
    # Combine core and level sequences
    all_seqs = CORE_SEQUENCES.copy()
    all_seqs.update(sequences_dict)
    
    seq_names = list(all_seqs.keys())
    n = len(seq_names)
    
    # Calculate identity matrix
    identity_matrix = np.zeros((n, n))
    for i, name1 in enumerate(seq_names):
        for j, name2 in enumerate(seq_names):
            identity = calculate_identity(all_seqs[name1], all_seqs[name2])
            identity_matrix[i, j] = identity
    
    # Convert identity to distance (100 - identity)
    distance_matrix = 100 - identity_matrix
    
    return distance_matrix, seq_names


def create_hierarchical_tree(distance_matrix, seq_names, 
                             output_file="phylogenetic_tree_hierarchical.png"):
    """Create hierarchical clustering dendrogram (publication quality)."""
    
    # Convert to condensed distance matrix for scipy
    condensed_dist = squareform(distance_matrix, checks=False)
    
    # Perform hierarchical clustering using UPGMA (average linkage)
    linkage_matrix = linkage(condensed_dist, method='average')
    
    # Create figure
    fig, ax = plt.subplots(figsize=(16, 12))
    
    # Custom colors for labels
    def get_label_color(label):
        if 'Ancestor' in label:
            return '#FFD700'  # Gold
        elif 'Human_WT' in label:
            return '#d62728'  # Red
        elif 'Mouse_WT' in label:
            return '#2ca02c'  # Green
        elif 'H2M' in label:
            return '#ff7f0e'  # Orange
        elif 'M2H' in label:
            return '#9467bd'  # Purple
        return 'black'
    
    # Create dendrogram
    dendro = dendrogram(
        linkage_matrix,
        labels=seq_names,
        ax=ax,
        orientation='right',
        color_threshold=0,
        above_threshold_color='#666666',
        leaf_font_size=9
    )
    
    # Color the labels
    ax_labels = ax.get_ymajorticklabels()
    for label in ax_labels:
        label.set_color(get_label_color(label.get_text()))
        if 'Ancestor' in label.get_text() or 'WT' in label.get_text():
            label.set_fontweight('bold')
            label.set_fontsize(11)
    
    # Formatting
    ax.set_xlabel('Evolutionary Distance (% divergence)', fontsize=14, fontweight='bold')
    ax.set_title('Phylogenetic Tree: GPX6 Evolution', 
                fontsize=18, fontweight='bold', pad=20)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#FFD700',
              markersize=10, label='Ancestor', markeredgecolor='black'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#d62728',
              markersize=10, label='Human WT (Sec)', markeredgecolor='black'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#2ca02c',
              markersize=10, label='Mouse WT (Cys)', markeredgecolor='black'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#ff7f0e',
              markersize=10, label='H→M Levels', markeredgecolor='black'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#9467bd',
              markersize=10, label='M→H Levels', markeredgecolor='black'),
    ]
    ax.legend(handles=legend_elements, loc='lower right', fontsize=11,
             framealpha=0.95, edgecolor='black')
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"✅ Hierarchical tree saved to: {output_file}")
    plt.close()


def create_circular_tree(distance_matrix, seq_names, 
                        output_file="phylogenetic_tree_circular.png"):
    """Create circular phylogenetic tree."""
    
    condensed_dist = squareform(distance_matrix, checks=False)
    linkage_matrix = linkage(condensed_dist, method='average')
    
    fig = plt.figure(figsize=(16, 16))
    ax = fig.add_subplot(111, projection='polar')
    
    # Get dendrogram data without plotting
    dendro_data = dendrogram(linkage_matrix, no_plot=True)
    
    # Get the order of leaves
    leaves_order = dendro_data['leaves']
    n_leaves = len(leaves_order)
    
    # Calculate angles for each leaf
    angles = np.linspace(0, 2*np.pi, n_leaves, endpoint=False)
    
    # Plot leaves
    for i, (leaf_idx, angle) in enumerate(zip(leaves_order, angles)):
        label = seq_names[leaf_idx]
        
        # Determine color
        if 'Ancestor' in label:
            color = '#FFD700'
            size = 200
            marker = '*'
        elif 'Human_WT' in label:
            color = '#d62728'
            size = 150
            marker = 's'
        elif 'Mouse_WT' in label:
            color = '#2ca02c'
            size = 150
            marker = 's'
        elif 'H2M' in label:
            color = '#ff7f0e'
            size = 50
            marker = 'o'
        elif 'M2H' in label:
            color = '#9467bd'
            size = 50
            marker = 'o'
        else:
            color = 'gray'
            size = 30
            marker = 'o'
        
        # Plot point
        ax.scatter(angle, 1.0, s=size, c=color, marker=marker, 
                  edgecolors='black', linewidths=1.5, zorder=10)
        
        # Add label
        label_dist = 1.15
        ha = 'left' if -np.pi/2 < angle < np.pi/2 else 'right'
        rotation = np.degrees(angle)
        if rotation > 90:
            rotation = rotation - 180
        
        fontsize = 10 if 'WT' in label or 'Ancestor' in label else 7
        fontweight = 'bold' if 'WT' in label or 'Ancestor' in label else 'normal'
        
        ax.text(angle, label_dist, label, rotation=rotation, 
               ha=ha, va='center', fontsize=fontsize, fontweight=fontweight)
    
    # Draw connecting lines (simplified - just to center)
    for angle in angles:
        ax.plot([angle, angle], [0, 1.0], 'k-', alpha=0.2, linewidth=0.5)
    
    # Formatting
    ax.set_ylim(0, 1.3)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['polar'].set_visible(False)
    ax.grid(False)
    
    plt.title('Circular Phylogenetic Tree: GPX6 Evolution\n', 
             fontsize=20, fontweight='bold', pad=30)
    
    # Legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], marker='*', color='w', markerfacecolor='#FFD700',
              markersize=15, label='Ancestor', markeredgecolor='black'),
        Line2D([0], [0], marker='s', color='w', markerfacecolor='#d62728',
              markersize=12, label='Human WT', markeredgecolor='black'),
        Line2D([0], [0], marker='s', color='w', markerfacecolor='#2ca02c',
              markersize=12, label='Mouse WT', markeredgecolor='black'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#ff7f0e',
              markersize=10, label='H→M Pathway', markeredgecolor='black'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#9467bd',
              markersize=10, label='M→H Pathway', markeredgecolor='black'),
    ]
    ax.legend(handles=legend_elements, loc='upper right', fontsize=12,
             framealpha=0.95, edgecolor='black', bbox_to_anchor=(1.3, 1.1))
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"✅ Circular tree saved to: {output_file}")
    plt.close()


def create_neighbor_joining_style_tree(distance_matrix, seq_names,
                                       output_file="phylogenetic_tree_publication.png"):
    """Create publication-quality tree with rooted structure."""
    
    fig, ax = plt.subplots(figsize=(18, 14))
    
    # Perform clustering
    condensed_dist = squareform(distance_matrix, checks=False)
    linkage_matrix = linkage(condensed_dist, method='average')
    dendro_data = dendrogram(linkage_matrix, no_plot=True)
    
    # Get ordering
    leaves_order = dendro_data['leaves']
    n_leaves = len(leaves_order)
    
    # Layout positions
    y_positions = {seq_names[i]: idx for idx, i in enumerate(leaves_order)}
    
    # Find key sequences
    ancestor_y = y_positions.get('Ancestor', n_leaves // 2)
    human_wt_y = y_positions.get('Human_WT', 0)
    mouse_wt_y = y_positions.get('Mouse_WT', n_leaves - 1)
    
    # Draw branches
    # Ancestor as root
    root_x = 0
    ax.scatter(root_x, ancestor_y, s=400, c='#FFD700', marker='*',
              edgecolors='black', linewidths=2, zorder=10, label='Ancestor (Root)')
    
    ax.text(root_x - 5, ancestor_y, 'Ancestor\n(Node 23)', 
           ha='right', va='center', fontsize=13, fontweight='bold',
           bbox=dict(boxstyle='round,pad=0.5', facecolor='#FFD700', 
                    edgecolor='black', linewidth=2, alpha=0.9))
    
    # Draw Human branch
    human_branch_x = 40
    ax.plot([root_x, human_branch_x], [ancestor_y, human_wt_y], 
           'r-', linewidth=3, alpha=0.8, label='Sec lineage')
    
    ax.scatter(human_branch_x, human_wt_y, s=300, c='#d62728', marker='s',
              edgecolors='black', linewidths=2, zorder=10)
    
    ax.text(human_branch_x + 2, human_wt_y, 'Human WT\n(Sec)\n87.3% to ancestor',
           ha='left', va='center', fontsize=11, fontweight='bold',
           bbox=dict(boxstyle='round,pad=0.5', facecolor='white',
                    edgecolor='#d62728', linewidth=2))
    
    # Draw Mouse branch
    ax.plot([root_x, human_branch_x], [ancestor_y, mouse_wt_y],
           'g--', linewidth=3, alpha=0.8, label='Cys lineage')
    
    ax.scatter(human_branch_x, mouse_wt_y, s=300, c='#2ca02c', marker='s',
              edgecolors='black', linewidths=2, zorder=10)
    
    ax.text(human_branch_x + 2, mouse_wt_y, 'Mouse WT\n(Cys)\n78.6% to ancestor',
           ha='left', va='center', fontsize=11, fontweight='bold',
           bbox=dict(boxstyle='round,pad=0.5', facecolor='white',
                    edgecolor='#2ca02c', linewidth=2))
    
    # Draw mutation levels
    # H2M levels (branching from Human)
    h2m_levels = [name for name in seq_names if 'H2M' in name]
    for i, level_name in enumerate(h2m_levels[:10]):  # Show first 10
        y = y_positions[level_name]
        level_x = human_branch_x + 15 + (i * 2)
        ax.plot([human_branch_x, level_x], [human_wt_y, y],
               color='#ff7f0e', linewidth=0.5, alpha=0.4, linestyle=':')
        ax.scatter(level_x, y, s=30, c='#ff7f0e', alpha=0.6, zorder=5)
    
    # M2H levels (branching from Mouse)
    m2h_levels = [name for name in seq_names if 'M2H' in name]
    for i, level_name in enumerate(m2h_levels[:10]):  # Show first 10
        y = y_positions[level_name]
        level_x = human_branch_x + 15 + (i * 2)
        ax.plot([human_branch_x, level_x], [mouse_wt_y, y],
               color='#9467bd', linewidth=0.5, alpha=0.4, linestyle=':')
        ax.scatter(level_x, y, s=30, c='#9467bd', alpha=0.6, zorder=5)
    
    # Formatting
    ax.set_xlim(-10, 80)
    ax.set_ylim(-1, n_leaves)
    ax.set_xlabel('Evolutionary Distance', fontsize=14, fontweight='bold')
    ax.set_title('Phylogenetic Tree: GPX6 Evolution with Mutation Pathways',
                fontsize=20, fontweight='bold', pad=20)
    ax.set_yticks([])
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Legend
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], marker='*', color='w', markerfacecolor='#FFD700',
              markersize=15, label='Ancestor (Root)', markeredgecolor='black', markeredgewidth=2),
        Line2D([0], [0], color='r', linewidth=3, label='Sec-containing lineage (Human)'),
        Line2D([0], [0], color='g', linewidth=3, linestyle='--', label='Cys-containing lineage (Mouse)'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#ff7f0e',
              markersize=8, label='H→M mutation levels', markeredgecolor='black'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#9467bd',
              markersize=8, label='M→H mutation levels', markeredgecolor='black'),
    ]
    ax.legend(handles=legend_elements, loc='best', fontsize=12,
             framealpha=0.95, edgecolor='black')
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"✅ Publication tree saved to: {output_file}")
    plt.close()


def main():
    """Main analysis function."""
    
    print("="*80)
    print("PHYLOGENETIC TREE GENERATION FROM SIMILARITY DATA")
    print("="*80)
    
    # Directories
    mouse_dir = "/home/hp/nayanika/github/GPX6/analysis/alignment/MOUSE"
    human_dir = "/home/hp/nayanika/github/GPX6/analysis/alignment/HUMAN"
    
    print(f"\n[1] Reading sequences from:")
    print(f"    Mouse: {mouse_dir}")
    print(f"    Human: {human_dir}")
    
    level_sequences = read_all_level_files(mouse_dir, human_dir)
    
    if level_sequences:
        print(f"\n    ✓ Loaded {len(level_sequences)} mutation level sequences")
    else:
        print("\n    ⚠️  Using only core sequences (Ancestor, Human_WT, Mouse_WT)")
    
    print("\n[2] Calculating distance matrix from sequence identities...")
    distance_matrix, seq_names = create_distance_matrix(level_sequences)
    print(f"    ✓ Created {len(seq_names)}x{len(seq_names)} distance matrix")
    
    print("\n[3] Creating phylogenetic trees...")
    
    print("    [3a] Hierarchical clustering tree (UPGMA)...")
    create_hierarchical_tree(distance_matrix, seq_names)
    
    print("    [3b] Circular phylogenetic tree...")
    create_circular_tree(distance_matrix, seq_names)
    
    print("    [3c] Publication-quality rooted tree...")
    create_neighbor_joining_style_tree(distance_matrix, seq_names)
    
    print("\n" + "="*80)
    print("✅ PHYLOGENETIC TREE GENERATION COMPLETE")
    print("="*80)
    print("\nGenerated files:")
    print("  • phylogenetic_tree_hierarchical.png")
    print("  • phylogenetic_tree_circular.png - Circular tree layout")
    print("  • phylogenetic_tree_publication.png - Publication-ready rooted tree")
    print("\nRecommendation for publication:")
    print("  - Use phylogenetic_tree_publication.png as main figure")
    print("  - Show phylogenetic_tree_hierarchical.png in supplementary")


if __name__ == "__main__":
    main()

PHYLOGENETIC TREE GENERATION FROM SIMILARITY DATA

[1] Reading sequences from:
    Mouse: /home/hp/nayanika/github/GPX6/analysis/alignment/MOUSE
    Human: /home/hp/nayanika/github/GPX6/analysis/alignment/HUMAN
  ✓ Read GPX6_level01.fasta
  ✓ Read GPX6_level02.fasta
  ✓ Read GPX6_level03.fasta
  ✓ Read GPX6_level04.fasta
  ✓ Read GPX6_level05.fasta
  ✓ Read GPX6_level06.fasta
  ✓ Read GPX6_level07.fasta
  ✓ Read GPX6_level08.fasta
  ✓ Read GPX6_level09.fasta
  ✓ Read GPX6_level10.fasta
  ✓ Read GPX6_level11.fasta
  ✓ Read GPX6_level12.fasta
  ✓ Read GPX6_level13.fasta
  ✓ Read GPX6_level14.fasta
  ✓ Read GPX6_level15.fasta
  ✓ Read GPX6_level16.fasta
  ✓ Read GPX6_level17.fasta
  ✓ Read GPX6_level18.fasta
  ✓ Read GPX6_level19.fasta
  ✓ Read GPX6_level20.fasta
  ✓ Read GPX6_level01.fasta
  ✓ Read GPX6_level02.fasta
  ✓ Read GPX6_level03.fasta
  ✓ Read GPX6_level04.fasta
  ✓ Read GPX6_level05.fasta
  ✓ Read GPX6_level06.fasta
  ✓ Read GPX6_level07.fasta
  ✓ Read GPX6_level08.fasta
  ✓ R

In [10]:
#!/usr/bin/env python3
"""
ULTIMATE Comprehensive Phylogenetic Analysis
Generates: Dendrograms, Trees, Heatmaps, Identity Matrix, BLOSUM62 scores
All-in-one publication-ready analysis
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
import seaborn as sns
from matplotlib.lines import Line2D
from pathlib import Path
import re


# BLOSUM62 Matrix
BLOSUM62 = {
    'A': {'A': 4, 'R': -1, 'N': -2, 'D': -2, 'C': 0, 'Q': -1, 'E': -1, 'G': 0, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 0, 'W': -3, 'Y': -2, 'V': 0},
    'R': {'A': -1, 'R': 5, 'N': 0, 'D': -2, 'C': -3, 'Q': 1, 'E': 0, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 2, 'M': -1, 'F': -3, 'P': -2, 'S': -1, 'T': -1, 'W': -3, 'Y': -2, 'V': -3},
    'N': {'A': -2, 'R': 0, 'N': 6, 'D': 1, 'C': -3, 'Q': 0, 'E': 0, 'G': 0, 'H': 1, 'I': -3, 'L': -3, 'K': 0, 'M': -2, 'F': -3, 'P': -2, 'S': 1, 'T': 0, 'W': -4, 'Y': -2, 'V': -3},
    'D': {'A': -2, 'R': -2, 'N': 1, 'D': 6, 'C': -3, 'Q': 0, 'E': 2, 'G': -1, 'H': -1, 'I': -3, 'L': -4, 'K': -1, 'M': -3, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -4, 'Y': -3, 'V': -3},
    'C': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': 9, 'Q': -3, 'E': -4, 'G': -3, 'H': -3, 'I': -1, 'L': -1, 'K': -3, 'M': -1, 'F': -2, 'P': -3, 'S': -1, 'T': -1, 'W': -2, 'Y': -2, 'V': -1},
    'Q': {'A': -1, 'R': 1, 'N': 0, 'D': 0, 'C': -3, 'Q': 5, 'E': 2, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 1, 'M': 0, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -2, 'Y': -1, 'V': -2},
    'E': {'A': -1, 'R': 0, 'N': 0, 'D': 2, 'C': -4, 'Q': 2, 'E': 5, 'G': -2, 'H': 0, 'I': -3, 'L': -3, 'K': 1, 'M': -2, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -2},
    'G': {'A': 0, 'R': -2, 'N': 0, 'D': -1, 'C': -3, 'Q': -2, 'E': -2, 'G': 6, 'H': -2, 'I': -4, 'L': -4, 'K': -2, 'M': -3, 'F': -3, 'P': -2, 'S': 0, 'T': -2, 'W': -2, 'Y': -3, 'V': -3},
    'H': {'A': -2, 'R': 0, 'N': 1, 'D': -1, 'C': -3, 'Q': 0, 'E': 0, 'G': -2, 'H': 8, 'I': -3, 'L': -3, 'K': -1, 'M': -2, 'F': -1, 'P': -2, 'S': -1, 'T': -2, 'W': -2, 'Y': 2, 'V': -3},
    'I': {'A': -1, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -3, 'E': -3, 'G': -4, 'H': -3, 'I': 4, 'L': 2, 'K': -3, 'M': 1, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -3, 'Y': -1, 'V': 3},
    'L': {'A': -1, 'R': -2, 'N': -3, 'D': -4, 'C': -1, 'Q': -2, 'E': -3, 'G': -4, 'H': -3, 'I': 2, 'L': 4, 'K': -2, 'M': 2, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -2, 'Y': -1, 'V': 1},
    'K': {'A': -1, 'R': 2, 'N': 0, 'D': -1, 'C': -3, 'Q': 1, 'E': 1, 'G': -2, 'H': -1, 'I': -3, 'L': -2, 'K': 5, 'M': -1, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -2},
    'M': {'A': -1, 'R': -1, 'N': -2, 'D': -3, 'C': -1, 'Q': 0, 'E': -2, 'G': -3, 'H': -2, 'I': 1, 'L': 2, 'K': -1, 'M': 5, 'F': 0, 'P': -2, 'S': -1, 'T': -1, 'W': -1, 'Y': -1, 'V': 1},
    'F': {'A': -2, 'R': -3, 'N': -3, 'D': -3, 'C': -2, 'Q': -3, 'E': -3, 'G': -3, 'H': -1, 'I': 0, 'L': 0, 'K': -3, 'M': 0, 'F': 6, 'P': -4, 'S': -2, 'T': -2, 'W': 1, 'Y': 3, 'V': -1},
    'P': {'A': -1, 'R': -2, 'N': -2, 'D': -1, 'C': -3, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -3, 'L': -3, 'K': -1, 'M': -2, 'F': -4, 'P': 7, 'S': -1, 'T': -1, 'W': -4, 'Y': -3, 'V': -2},
    'S': {'A': 1, 'R': -1, 'N': 1, 'D': 0, 'C': -1, 'Q': 0, 'E': 0, 'G': 0, 'H': -1, 'I': -2, 'L': -2, 'K': 0, 'M': -1, 'F': -2, 'P': -1, 'S': 4, 'T': 1, 'W': -3, 'Y': -2, 'V': -2},
    'T': {'A': 0, 'R': -1, 'N': 0, 'D': -1, 'C': -1, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 5, 'W': -2, 'Y': -2, 'V': 0},
    'W': {'A': -3, 'R': -3, 'N': -4, 'D': -4, 'C': -2, 'Q': -2, 'E': -3, 'G': -2, 'H': -2, 'I': -3, 'L': -2, 'K': -3, 'M': -1, 'F': 1, 'P': -4, 'S': -3, 'T': -2, 'W': 11, 'Y': 2, 'V': -3},
    'Y': {'A': -2, 'R': -2, 'N': -2, 'D': -3, 'C': -2, 'Q': -1, 'E': -2, 'G': -3, 'H': 2, 'I': -1, 'L': -1, 'K': -2, 'M': -1, 'F': 3, 'P': -3, 'S': -2, 'T': -2, 'W': 2, 'Y': 7, 'V': -1},
    'V': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -2, 'E': -2, 'G': -3, 'H': -3, 'I': 3, 'L': 1, 'K': -2, 'M': 1, 'F': -1, 'P': -2, 'S': -2, 'T': 0, 'W': -3, 'Y': -1, 'V': 4}
}

# Core sequences
CORE_SEQUENCES = {
    'Ancestor': 'SQKMKMDCYKGVTGTIYEYGALTLNGEEYIQFKQYAGKHVLFINVATYGLTAQYPELNALQEELKPFGVVLLGFPCNQFGKQEPGKNSEILSGLKYVRPGGGFVPNFQLFEKGDVNGEKEQKVFTFLKNSCPPTSDLLGSPKQLFWEPMKVHDIRWNFEKFLVGPDGVPVMRWFHRASVSTVKSDILEYLKQFTPE',
    'Mouse_WT': 'PQKSKVDXNKGVTGTVYEYGANTIDGGEFVNFQQYAGKXILFVNVASFCGLTATYPELNTLQEELKPFNVTVLGFPCNQFGKQEPGKNSEILLGLKYVRPGGGYVPNFQLFEKGDVNGDNEQKVFSFLKNSXPPTSELFGSPEXLFWDPMKVXDIRWNFEKFLVGPDGVPVMRWFXXTPVRIVQSDIMEYLNQTS',
    'Human_WT': 'PQNRKVDXNKGVTGTIYEYGALTLNGEEYIQFKQFAGKXVLFVNVAAYUGLAAQYPELNALQEELKNFGVIVLAFPCNQFGKQEPGTNSEILLGLKYVCPGSGFVPSFQLFEKGDVNGEKEQKVFTFLKNSXPPTSDLLGSSSQLFWEPMKVXDIRWNFEKFLVGPDGVPVMXWFXQAPVSTVKSDILEYLKQFNTX'
}


def read_fasta_file(filepath):
    """Read a FASTA file and return sequences."""
    sequences = {}
    current_name = None
    current_seq = []
    
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                
                if line.startswith('>'):
                    if current_name:
                        sequences[current_name] = ''.join(current_seq)
                    current_name = line[1:].strip()
                    current_seq = []
                else:
                    current_seq.append(line.upper())
            
            if current_name:
                sequences[current_name] = ''.join(current_seq)
    except Exception as e:
        return {}
    
    return sequences


def read_all_level_files(mouse_dir, human_dir):
    """Read all level files from both directories."""
    all_sequences = {}
    
    mouse_path = Path(mouse_dir)
    if mouse_path.exists():
        fasta_files = list(mouse_path.glob('GPX6_level*.fasta')) + list(mouse_path.glob('level*.fasta'))
        for fasta_file in sorted(set(fasta_files)):
            level = re.search(r'level(\d+)', fasta_file.name)
            if level:
                level_num = int(level.group(1))
                seqs = read_fasta_file(fasta_file)
                for name, seq in seqs.items():
                    key = f"M2H_Level{level_num:02d}"
                    all_sequences[key] = seq
                print(f"  ✓ Mouse {fasta_file.name}")
    
    human_path = Path(human_dir)
    if human_path.exists():
        fasta_files = list(human_path.glob('GPX6_level*.fasta')) + list(human_path.glob('level*.fasta'))
        for fasta_file in sorted(set(fasta_files)):
            level = re.search(r'level(\d+)', fasta_file.name)
            if level:
                level_num = int(level.group(1))
                seqs = read_fasta_file(fasta_file)
                for name, seq in seqs.items():
                    key = f"H2M_Level{level_num:02d}"
                    all_sequences[key] = seq
                print(f"  ✓ Human {fasta_file.name}")
    
    return all_sequences


def calculate_identity(seq1, seq2):
    """Calculate sequence identity."""
    matches = 0
    total = 0
    
    for aa1, aa2 in zip(seq1, seq2):
        if aa1 in '-XU*' or aa2 in '-XU*':
            continue
        total += 1
        if aa1 == aa2:
            matches += 1
    
    return (matches / total * 100) if total > 0 else 0.0


def calculate_blosum_score(seq1, seq2, normalized=True):
    """Calculate BLOSUM62 score."""
    score = 0
    comparisons = 0
    
    for aa1, aa2 in zip(seq1, seq2):
        if aa1 in '-XU*' or aa2 in '-XU*':
            continue
        
        if aa1 in BLOSUM62 and aa2 in BLOSUM62[aa1]:
            score += BLOSUM62[aa1][aa2]
            comparisons += 1
    
    if normalized and comparisons > 0:
        return score / comparisons
    return score


def create_all_matrices(sequences_dict):
    """Create all matrices (identity, BLOSUM, distance)."""
    all_seqs = CORE_SEQUENCES.copy()
    all_seqs.update(sequences_dict)
    
    seq_names = list(all_seqs.keys())
    n = len(seq_names)
    
    identity_matrix = np.zeros((n, n))
    blosum_matrix = np.zeros((n, n))
    
    for i, name1 in enumerate(seq_names):
        for j, name2 in enumerate(seq_names):
            seq1 = all_seqs[name1]
            seq2 = all_seqs[name2]
            
            identity_matrix[i, j] = calculate_identity(seq1, seq2)
            blosum_matrix[i, j] = calculate_blosum_score(seq1, seq2)
    
    # Distance matrix for clustering
    distance_matrix = 100 - identity_matrix
    
    return identity_matrix, blosum_matrix, distance_matrix, seq_names


def create_comprehensive_heatmaps(identity_matrix, blosum_matrix, seq_names,
                                  output_file="comprehensive_heatmaps.png"):
    """Create side-by-side heatmaps."""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(22, 11))
    
    # Identity heatmap
    im1 = ax1.imshow(identity_matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=100)
    ax1.set_xticks(range(len(seq_names)))
    ax1.set_yticks(range(len(seq_names)))
    ax1.set_xticklabels(seq_names, rotation=90, ha='right', fontsize=8)
    ax1.set_yticklabels(seq_names, fontsize=8)
    ax1.set_title('Sequence Identity Matrix (%)', fontsize=18, fontweight='bold', pad=20)
    
    cbar1 = plt.colorbar(im1, ax=ax1)
    cbar1.set_label('Identity (%)', fontsize=14, rotation=270, labelpad=25)
    
    # BLOSUM heatmap
    im2 = ax2.imshow(blosum_matrix, cmap='RdBu_r', aspect='auto')
    ax2.set_xticks(range(len(seq_names)))
    ax2.set_yticks(range(len(seq_names)))
    ax2.set_xticklabels(seq_names, rotation=90, ha='right', fontsize=8)
    ax2.set_yticklabels(seq_names, fontsize=8)
    ax2.set_title('BLOSUM62 Similarity Matrix', fontsize=18, fontweight='bold', pad=20)
    
    cbar2 = plt.colorbar(im2, ax=ax2)
    cbar2.set_label('BLOSUM Score (normalized)', fontsize=14, rotation=270, labelpad=25)
    
    plt.suptitle('Comprehensive Sequence Similarity Analysis', 
                fontsize=20, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"✅ Comprehensive heatmaps saved to: {output_file}")
    plt.close()


def create_upgma_dendrogram(distance_matrix, seq_names,
                            output_file="phylogenetic_dendrogram_UPGMA.png"):
    """Create UPGMA clustering dendrogram."""
    
    condensed_dist = squareform(distance_matrix, checks=False)
    linkage_matrix = linkage(condensed_dist, method='average')
    
    fig, ax = plt.subplots(figsize=(18, 14))
    
    def get_label_color(label):
        if 'Ancestor' in label:
            return '#FFD700'
        elif 'Human_WT' in label:
            return '#d62728'
        elif 'Mouse_WT' in label:
            return '#2ca02c'
        elif 'H2M' in label:
            return '#ff7f0e'
        elif 'M2H' in label:
            return '#9467bd'
        return 'black'
    
    dendro = dendrogram(
        linkage_matrix,
        labels=seq_names,
        ax=ax,
        orientation='right',
        color_threshold=0,
        above_threshold_color='#666666',
        leaf_font_size=9
    )
    
    ax_labels = ax.get_ymajorticklabels()
    for label in ax_labels:
        label.set_color(get_label_color(label.get_text()))
        if 'Ancestor' in label.get_text() or 'WT' in label.get_text():
            label.set_fontweight('bold')
            label.set_fontsize(11)
    
    ax.set_xlabel('Evolutionary Distance (% divergence)', fontsize=16, fontweight='bold')
    ax.set_title('UPGMA Phylogenetic Tree: GPX6 Evolution', 
                fontsize=20, fontweight='bold', pad=20)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#FFD700',
              markersize=12, label='Ancestor', markeredgecolor='black'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#d62728',
              markersize=12, label='Human WT (Sec)', markeredgecolor='black'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#2ca02c',
              markersize=12, label='Mouse WT (Cys)', markeredgecolor='black'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#ff7f0e',
              markersize=10, label='H→M Levels', markeredgecolor='black'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#9467bd',
              markersize=10, label='M→H Levels', markeredgecolor='black'),
    ]
    ax.legend(handles=legend_elements, loc='lower right', fontsize=12,
             framealpha=0.95, edgecolor='black')
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"✅ UPGMA dendrogram saved to: {output_file}")
    plt.close()


def create_publication_tree(distance_matrix, seq_names,
                            output_file="phylogenetic_tree_publication.png"):
    """Create publication-ready rooted phylogenetic tree."""
    
    condensed_dist = squareform(distance_matrix, checks=False)
    linkage_matrix = linkage(condensed_dist, method='average')
    dendro_data = dendrogram(linkage_matrix, no_plot=True)
    
    fig, ax = plt.subplots(figsize=(20, 16))
    
    leaves_order = dendro_data['leaves']
    n_leaves = len(leaves_order)
    y_positions = {seq_names[i]: idx for idx, i in enumerate(leaves_order)}
    
    ancestor_y = y_positions.get('Ancestor', n_leaves // 2)
    human_wt_y = y_positions.get('Human_WT', 0)
    mouse_wt_y = y_positions.get('Mouse_WT', n_leaves - 1)
    
    root_x = 0
    ax.scatter(root_x, ancestor_y, s=600, c='#FFD700', marker='*',
              edgecolors='black', linewidths=3, zorder=10)
    
    ax.text(root_x - 5, ancestor_y, 'Ancestor\n(Node 23)', 
           ha='right', va='center', fontsize=15, fontweight='bold',
           bbox=dict(boxstyle='round,pad=0.7', facecolor='#FFD700', 
                    edgecolor='black', linewidth=2, alpha=0.95))
    
    human_branch_x = 50
    ax.plot([root_x, human_branch_x], [ancestor_y, human_wt_y], 
           'r-', linewidth=4, alpha=0.8)
    ax.scatter(human_branch_x, human_wt_y, s=500, c='#d62728', marker='s',
              edgecolors='black', linewidths=3, zorder=10)
    ax.text(human_branch_x + 3, human_wt_y, 'Human WT\n(Sec)\n87.3% identity',
           ha='left', va='center', fontsize=13, fontweight='bold',
           bbox=dict(boxstyle='round,pad=0.7', facecolor='white',
                    edgecolor='#d62728', linewidth=3))
    
    ax.plot([root_x, human_branch_x], [ancestor_y, mouse_wt_y],
           'g--', linewidth=4, alpha=0.8)
    ax.scatter(human_branch_x, mouse_wt_y, s=500, c='#2ca02c', marker='s',
              edgecolors='black', linewidths=3, zorder=10)
    ax.text(human_branch_x + 3, mouse_wt_y, 'Mouse WT\n(Cys)\n78.6% identity',
           ha='left', va='center', fontsize=13, fontweight='bold',
           bbox=dict(boxstyle='round,pad=0.7', facecolor='white',
                    edgecolor='#2ca02c', linewidth=3))
    
    # Draw mutation levels
    h2m_levels = [name for name in seq_names if 'H2M' in name]
    for level_name in h2m_levels:
        y = y_positions[level_name]
        level_x = human_branch_x + 20
        ax.plot([human_branch_x, level_x], [human_wt_y, y],
               color='#ff7f0e', linewidth=0.8, alpha=0.4, linestyle=':')
        ax.scatter(level_x, y, s=40, c='#ff7f0e', alpha=0.7, zorder=5,
                  edgecolors='black', linewidths=0.5)
    
    m2h_levels = [name for name in seq_names if 'M2H' in name]
    for level_name in m2h_levels:
        y = y_positions[level_name]
        level_x = human_branch_x + 20
        ax.plot([human_branch_x, level_x], [mouse_wt_y, y],
               color='#9467bd', linewidth=0.8, alpha=0.4, linestyle=':')
        ax.scatter(level_x, y, s=40, c='#9467bd', alpha=0.7, zorder=5,
                  edgecolors='black', linewidths=0.5)
    
    ax.set_xlim(-10, 90)
    ax.set_ylim(-2, n_leaves + 1)
    ax.set_xlabel('Evolutionary Distance', fontsize=16, fontweight='bold')
    ax.set_title('Publication-Quality Phylogenetic Tree: GPX6 Evolution',
                fontsize=22, fontweight='bold', pad=25)
    ax.set_yticks([])
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    legend_elements = [
        Line2D([0], [0], marker='*', color='w', markerfacecolor='#FFD700',
              markersize=18, label='Ancestor (Node 23)', markeredgecolor='black', markeredgewidth=2),
        Line2D([0], [0], color='r', linewidth=4, label='Sec lineage (Human)'),
        Line2D([0], [0], color='g', linewidth=4, linestyle='--', label='Cys lineage (Mouse)'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#ff7f0e',
              markersize=10, label='H→M mutation levels', markeredgecolor='black'),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#9467bd',
              markersize=10, label='M→H mutation levels', markeredgecolor='black'),
    ]
    ax.legend(handles=legend_elements, loc='best', fontsize=13,
             framealpha=0.95, edgecolor='black', fancybox=True, shadow=True)
    
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"✅ Publication tree saved to: {output_file}")
    plt.close()


def create_summary_report(identity_matrix, blosum_matrix, seq_names,
                         output_file="comprehensive_analysis_summary.txt"):
    """Create comprehensive text summary."""
    
    ancestor_idx = seq_names.index('Ancestor') if 'Ancestor' in seq_names else None
    human_idx = seq_names.index('Human_WT') if 'Human_WT' in seq_names else None
    mouse_idx = seq_names.index('Mouse_WT') if 'Mouse_WT' in seq_names else None
    
    with open(output_file, 'w') as f:
        f.write("="*90 + "\n")
        f.write("COMPREHENSIVE PHYLOGENETIC ANALYSIS SUMMARY\n")
        f.write("="*90 + "\n\n")
        
        f.write("CORE SEQUENCES - IDENTITY TO ANCESTOR:\n")
        f.write("-"*90 + "\n")
        if ancestor_idx is not None:
            if human_idx is not None:
                f.write(f"Human WT (Sec):  {identity_matrix[human_idx, ancestor_idx]:6.2f}%\n")
            if mouse_idx is not None:
                f.write(f"Mouse WT (Cys):  {identity_matrix[mouse_idx, ancestor_idx]:6.2f}%\n")
        f.write("\n")
        
        f.write("CORE SEQUENCES - BLOSUM62 SCORES TO ANCESTOR:\n")
        f.write("-"*90 + "\n")
        if ancestor_idx is not None:
            if human_idx is not None:
                f.write(f"Human WT:  {blosum_matrix[human_idx, ancestor_idx]:7.3f} (normalized)\n")
            if mouse_idx is not None:
                f.write(f"Mouse WT:  {blosum_matrix[mouse_idx, ancestor_idx]:7.3f} (normalized)\n")
        f.write("\n")
        
        if ancestor_idx is not None:
            f.write("MUTATION LEVELS - IDENTITY TO ANCESTOR:\n")
            f.write("-"*90 + "\n\n")
            
            f.write("Human → Mouse Pathway:\n")
            h2m_levels = [(name, i) for i, name in enumerate(seq_names) if 'H2M_' in name]
            for name, idx in sorted(h2m_levels):
                identity = identity_matrix[idx, ancestor_idx]
                blosum = blosum_matrix[idx, ancestor_idx]
                f.write(f"  {name:20} Identity: {identity:6.2f}%  BLOSUM: {blosum:7.3f}\n")
            
            f.write("\nMouse → Human Pathway:\n")
            m2h_levels = [(name, i) for i, name in enumerate(seq_names) if 'M2H_' in name]
            for name, idx in sorted(m2h_levels):
                identity = identity_matrix[idx, ancestor_idx]
                blosum = blosum_matrix[idx, ancestor_idx]
                f.write(f"  {name:20} Identity: {identity:6.2f}%  BLOSUM: {blosum:7.3f}\n")
        
        f.write("\n" + "="*90 + "\n")
    
    print(f"✅ Summary report saved to: {output_file}")


def main():
    """Ultimate comprehensive analysis."""
    
    print("="*90)
    print("ULTIMATE COMPREHENSIVE PHYLOGENETIC ANALYSIS")
    print("Generates: Heatmaps, Dendrogram, Publication Tree, Summary")
    print("="*90)
    
    # Directories
    mouse_dir = "/home/hp/nayanika/github/GPX6/analysis/alignment/MOUSE"
    human_dir = "/home/hp/nayanika/github/GPX6/analysis/alignment/HUMAN"
    
    print(f"\n[1] Reading FASTA files...")
    level_sequences = read_all_level_files(mouse_dir, human_dir)
    
    if not level_sequences:
        print("\n⚠️  Using only core sequences (Ancestor, Human_WT, Mouse_WT)")
    else:
        print(f"\n✓ Loaded {len(level_sequences)} mutation level sequences")
    
    print("\n[2] Calculating matrices...")
    identity_matrix, blosum_matrix, distance_matrix, seq_names = create_all_matrices(level_sequences)
    print(f"✓ Created {len(seq_names)}x{len(seq_names)} matrices")
    
    print("\n[3] Creating visualizations...")
    print("    [3a] Comprehensive heatmaps (Identity + BLOSUM62)...")
    create_comprehensive_heatmaps(identity_matrix, blosum_matrix, seq_names)
    
    print("    [3b] UPGMA clustering dendrogram...")
    create_upgma_dendrogram(distance_matrix, seq_names)
    
    print("    [3c] Publication-quality phylogenetic tree...")
    create_publication_tree(distance_matrix, seq_names)
    
    print("\n[4] Generating comprehensive summary...")
    create_summary_report(identity_matrix, blosum_matrix, seq_names)
    
    print("\n" + "="*90)
    print("✅ COMPLETE ANALYSIS FINISHED")
    print("="*90)
    print("\n📊 Generated Files:")
    print("  1. comprehensive_heatmaps.png - Identity & BLOSUM62 matrices")
    print("  2. phylogenetic_dendrogram_UPGMA.png - Clustering dendrogram")
    print("  3. phylogenetic_tree_publication.png - Publication-ready tree")
    print("  4. comprehensive_analysis_summary.txt - Full summary with all scores")
    print("\n💡 For Publication:")
    print("  • Main Figure: phylogenetic_tree_publication.png")
    print("  • Supplementary: phylogenetic_dendrogram_UPGMA.png")
    print("  • Supplementary: comprehensive_heatmaps.png")
    print("  • Table: comprehensive_analysis_summary.txt")


if __name__ == "__main__":
    main()

ULTIMATE COMPREHENSIVE PHYLOGENETIC ANALYSIS
Generates: Heatmaps, Dendrogram, Publication Tree, Summary

[1] Reading FASTA files...
  ✓ Mouse GPX6_level01.fasta
  ✓ Mouse GPX6_level02.fasta
  ✓ Mouse GPX6_level03.fasta
  ✓ Mouse GPX6_level04.fasta
  ✓ Mouse GPX6_level05.fasta
  ✓ Mouse GPX6_level06.fasta
  ✓ Mouse GPX6_level07.fasta
  ✓ Mouse GPX6_level08.fasta
  ✓ Mouse GPX6_level09.fasta
  ✓ Mouse GPX6_level10.fasta
  ✓ Mouse GPX6_level11.fasta
  ✓ Mouse GPX6_level12.fasta
  ✓ Mouse GPX6_level13.fasta
  ✓ Mouse GPX6_level14.fasta
  ✓ Mouse GPX6_level15.fasta
  ✓ Mouse GPX6_level16.fasta
  ✓ Mouse GPX6_level17.fasta
  ✓ Mouse GPX6_level18.fasta
  ✓ Mouse GPX6_level19.fasta
  ✓ Mouse GPX6_level20.fasta
  ✓ Human GPX6_level01.fasta
  ✓ Human GPX6_level02.fasta
  ✓ Human GPX6_level03.fasta
  ✓ Human GPX6_level04.fasta
  ✓ Human GPX6_level05.fasta
  ✓ Human GPX6_level06.fasta
  ✓ Human GPX6_level07.fasta
  ✓ Human GPX6_level08.fasta
  ✓ Human GPX6_level09.fasta
  ✓ Human GPX6_level10.fast